In [331]:
import datetime
import os
import re
import time
from urllib.parse import urlencode
from urllib.request import urlopen

import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

from src.constants._master import Master
from src.constants._results_cols import ResultsCols as Cols
from src.preparing.DataLoader import DataLoader
from src.preparingdfprepare_chrome_driver import prepare_chrome_driver

%load_ext autoreload

ModuleNotFoundError: No module named 'src.preparingdfprepare_chrome_driver'

In [106]:
from src.preparing.table_creator import TableCreator
from src.preparing.url_loader import KaisaiDateLoader
from tqdm import tqdm
import os
import pandas as pd
from bs4 import BeautifulSoup


def create_race_results_table_():
    race_html_list = ["202001010101.bin"]#, "202001010102.bin", "202001010103.bin"]

    print("creating race_results_table")

    for race_html in tqdm(race_html_list):
        race_html_path = os.path.join("./data/html/race/", race_html)
        with open(race_html_path, "rb") as f:
            # 保存してあるbinファイルを読み込む
            html = f.read()
            # メインとなるレース結果テーブルデータを取得
            df = pd.read_html(html)
            print("df:", df)
            # htmlをsoupオブジェクトに変換
            soup = BeautifulSoup(html, "lxml")
            print("soup: ", soup)


alias = "race_results_table"
data_request = TableCreator()
data_request.set_args(alias)
data_request.pre_process_display()
create_race_results_table_()

race_results_table
self.from_location:  https://db.netkeiba.com/race/
to_temp_location:  ./data/tmp/race_results/
to_location:  ./data/html/race_results/
self.from_local_location:  ./data/html/race/
self.from_local_file_name:  *.bin
batch_size: 100
reloaded_*.bin type:  *.bin
race_results_table
self.from_location:  https://db.netkeiba.com/race/
to_temp_location:  ./data/tmp/race_results/
to_location:  ./data/html/race_results/
self.from_local_location:  ./data/html/race/
self.from_local_file_name:  *.bin
batch_size: 100
reloaded_*.bin type:  *.bin
creating race_results_table


100%|██████████| 1/1 [00:00<00:00,  6.62it/s]

df: [   着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差    単勝  人 気      馬体重  \
0    1    6    6     ウインルーア  牝2  54  横山武史  1:49.7    NaN  16.0    3  438(+4)   
1    2    2    2     アークライト  牡2  54  ルメール  1:50.0  1.3/4   1.9    2   510(0)   
2    3    3    3  ギャラントウォリア  牡2  54  池添謙一  1:50.1    1/2   1.8    1  482(-6)   
3    4    1    1     ジュンブーケ  牝2  52  亀田温心  1:50.5  2.1/2  22.2    4   442(0)   
4    5    4    4    キタノマンゲツ  牡2  54  藤岡康太  1:51.0      3  55.7    5  426(-8)   
5    6    5    5     アイフレンズ  牝2  54   黛弘人  1:52.7      大  73.9    6  426(-2)   

        調教師  
0  [東] 青木孝文  
1  [東] 藤沢和雄  
2   [西] 平田修  
3   [西] 森秀行  
4  [西] 中尾秀正  
5  [東] 中野栄治  ,     0      1        2    3
0  単勝      6     1600    3
1  複勝    6 2  490 130  4 2
2  馬連  2 - 6     1410    4,      0                  1            2      3
0  ワイド  2 - 6 3 - 6 2 - 3  150 170 110  3 4 1
1   馬単              6 → 2         3920      9
2  三連複          2 - 3 - 6          240      2
3  三連単          6 → 2 → 3         5180    

In [31]:
# from src.preparing.table_creator import TableCreator
# from src.preparing.url_loader import KaisaiDateLoader
from tqdm import tqdm
import os
import pandas as pd
from bs4 import BeautifulSoup
import re
from src.constants._master import Master


def create_race_info_table_():
    race_html_list = ["202001010101.bin", "202001010102.bin", "202001010103.bin"]

    print("creating race_results_table")

    for race_html in tqdm(race_html_list):
        race_html_path = os.path.join("./data/html/race/", race_html)
        with open(race_html_path, "rb") as f:
            # 保存してあるbinファイルを読み込む
            html = f.read()
            #df = pd.read_html(html)
            # print("df:",df)
            # htmlをsoupオブジェクトに変換
            soup = BeautifulSoup(html, "lxml")
            # print("soup: ",soup)
            # 天候、レースの種類、コースの長さ、馬場の状態、日付、回り、レースクラスをスクレイピング
            texts = (
                soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
            )
            # print("texts: ",texts)
            info = re.findall(r"\w+", texts)
            # print("info: ",info)

            df = pd.DataFrame()
            # 障害レースフラグを初期化
            hurdle_race_flg = False
            for text in info:
                if text in ["芝", "ダート"]:
                    df["race_type"] = [text]
                if "障" in text:
                    df["race_type"] = ["障害"]
                    hurdle_race_flg = True
                if "m" in text:
                    # 20211212：[0]→[-1]に修正
                    df["course_len"] = [int(re.findall(r"\d+", text)[-1])]
                if text in Master.GROUND_STATE_LIST:
                    df["ground_state"] = [text]
                if text in Master.WEATHER_LIST:
                    df["weather"] = [text]
                if "年" in text:
                    df["date"] = [text]
                if "右" in text:
                    df["around"] = [Master.AROUND_LIST[0]]
                if "左" in text:
                    df["around"] = [Master.AROUND_LIST[1]]
                if "直線" in text:
                    df["around"] = [Master.AROUND_LIST[2]]
                if "新馬" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[0]]
                if "未勝利" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[1]]
                if ("1勝クラス" in text) or ("500万下" in text):
                    df["race_class"] = [Master.RACE_CLASS_LIST[2]]
                if ("2勝クラス" in text) or ("1000万下" in text):
                    df["race_class"] = [Master.RACE_CLASS_LIST[3]]
                if ("3勝クラス" in text) or ("1600万下" in text):
                    df["race_class"] = [Master.RACE_CLASS_LIST[4]]
                if "オープン" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[5]]
            # グレードレース情報の取得
            grade_text = soup.find("div", attrs={"class": "data_intro"}).find_all("h1")[0].text
            if "G3" in grade_text:
                df["race_class"] = [Master.RACE_CLASS_LIST[6]] * len(df)
            elif "G2" in grade_text:
                df["race_class"] = [Master.RACE_CLASS_LIST[7]] * len(df)
            elif "G1" in grade_text:
                df["race_class"] = [Master.RACE_CLASS_LIST[8]] * len(df)

            # 障害レースの場合
            if hurdle_race_flg:
                df["around"] = [Master.AROUND_LIST[3]]
                df["race_class"] = [Master.RACE_CLASS_LIST[9]]

            # インデックスをrace_idにする
            race_id = re.findall(r"(\d+).bin", race_html)[0]
            df.index = [race_id] * len(df)
            race_infos = {}
            race_infos[race_id] = df
    race_infos_df = pd.concat([race_infos[key] for key in race_infos])
    print(race_infos_df)


# alias = "race_results_table"
# data_request = TableCreator()
# data_request.set_args(alias)
# data_request.pre_process_display()
create_race_info_table_()

creating race_results_table


100%|██████████| 3/3 [00:00<00:00, 15.91it/s]

              course_len around weather race_type ground_state        date  \
202001010103        1500      右       曇         芝            良  2020年7月25日   

             race_class  
202001010103        未勝利  


In [65]:
import time
from urllib.request import urlopen

def scrape_horse_id_list_():
    """
    当日出走するhorse_id一覧を取得
    """

    print("scraping horse_id_list")
    race_id_list =["202001010101","202001010102", "202001010103"]
    horse_id_list=[]
    for race_id in tqdm(race_id_list):
        # wait = WebDriverWait(driver, waiting_time)
        try:
            url = "https://db.netkeiba.com/race/"+ race_id

            # wait.until(EC.presence_of_element_located((By.CLASS_NAME, "HorseInfo")))
            html = urlopen(url)
            #soup = BeautifulSoup(html, "lxml", from_encoding="utf-8")
            soup = BeautifulSoup(html, "lxml")
            #print("soup",soup)
            horse_td_list = soup.find_all("td", attrs={"class": "txt_l"})
            horse_ids =[a['href'] for td in horse_td_list if (a := td.find('a')) and '/horse/' in a['href']]
            horse_id = [re.search(r'/horse/(\d+)/', href).group(1)  for href in horse_ids]
            horse_id_list.append(horse_id)
            # 内側のリストを外す
            flat_list = [item for sublist in horse_id_list for item in sublist]
            target_data = sorted(set(flat_list))
        except Exception as e:
            print(e)
            break
    print("horse_td_list:",target_data)



scrape_horse_id_list_()

scraping horse_id_list


100%|██████████| 3/3 [00:01<00:00,  2.49it/s]

horse_td_list: ['2017100432', '2017100813', '2017101364', '2017101757', '2017102190', '2017102915', '2017102918', '2017103355', '2017103622', '2017103709', '2017103841', '2017103915', '2017103953', '2017103996', '2017104065', '2017104160', '2017104334', '2017104792', '2017104859', '2017104882', '2017105136', '2017105153', '2017105242', '2017105261', '2017105324', '2017106065', '2017106103', '2017106389', '2018100828', '2018101626', '2018102410', '2018104800', '2018105193', '2018106434']


In [93]:
import datetime
import os
import re
import time
from urllib.parse import urlencode
from urllib.request import urlopen

import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

from src.preparing.DataLoader import DataLoader
from src.preparing.prepare_chrome_driver import prepare_chrome_driver

def scrape_peds_list_():
    """
    horse/pedページのhtmlを受け取って、血統のDataFrameに変換する関数。
    """
    # skip対象ではないゴミファイルの掃除

    ped_html_list = ["2018102657.bin"]#,"2017100282.bin"]
    data_index = 1
    peds = {}
    print("preparing peds table")
    for ped_html in tqdm(ped_html_list):
        ped_html_path = os.path.join("./data/html/ped/", ped_html)
        # print("ped_html_path", ped_html_path)
        try:
            with open(ped_html_path, "rb") as f:
                # 保存してあるbinファイルを読み込む
                html = f.read()
                # horse_idを取得
                #print("ped_html",ped_html)
                horse_id = re.findall(r"(\d+)", ped_html)[0]
                print("horse_id",horse_id)
                # htmlをsoupオブジェクトに変換
                soup = BeautifulSoup(html, "lxml")
                #print("soup",soup)
                peds_id_list = []
                # 血統データからhorse_idを取得する
                horse_a_list = soup.find("table", attrs={"summary": "5代血統表"}).find_all(
                    "a", attrs={"href": re.compile(r"^/horse/\w{10}")}
                )

                for a in horse_a_list:
                    # 血統データのhorse_idを抜き出す
                    work_peds_id = re.findall(r"horse\W(\w{10})", a["href"])[0]
                    print("work_peds_id",work_peds_id)
                    peds_id_list.append(work_peds_id)
                print("peds_id_list",peds_id_list)
                peds[horse_id] = peds_id_list
                print("peds",peds[horse_id] )
                # pd.DataFrame型にして一つのデータにまとめて、列と行の入れ替えして、列名をpeds_0, ..., peds_61にする
                peds_df = pd.DataFrame.from_dict(peds, orient="index").add_prefix("peds_")
                print("peds_df",peds_df)
            #if data_index % self.batch_size == 0:
            #    self.target_data = peds_df
            #    self.save_temp_file("peds_list")
            #    self.obtained_last_key = ped_html
        except Exception as e:
            print("Error at {}: {}".format(ped_html, e))
        #data_index = +1

scrape_peds_list_()

preparing peds table


100%|██████████| 1/1 [00:00<00:00, 17.78it/s]

horse_id 2018102657
work_peds_id 2012104511
work_peds_id 2001103460
work_peds_id 000a001d7e
work_peds_id 000a001607
work_peds_id 000a000e46
work_peds_id 000a007e0c
work_peds_id 000a009961
work_peds_id 000a001676
work_peds_id 000a0084c9
work_peds_id 000a00680e
work_peds_id 000a0001d6
work_peds_id 000a000378
work_peds_id 000a009241
work_peds_id 000a00b1e0
work_peds_id 000a000e1d
work_peds_id 000a009608
work_peds_id 2000107542
work_peds_id 000a00033a
work_peds_id 000a0012bf
work_peds_id 000a000f2b
work_peds_id 000a007459
work_peds_id 000a008c1e
work_peds_id 000a0019b6
work_peds_id 000a008c1d
work_peds_id 1993109154
work_peds_id 1983109006
work_peds_id 000a000827
work_peds_id 000a00859c
work_peds_id 1980101983
work_peds_id 000a000258
work_peds_id 1955105713
work_peds_id 2003101283
work_peds_id 000a000134
work_peds_id 000a001607
work_peds_id 000a000e46
work_peds_id 000a000f89
work_peds_id 000a007984
work_peds_id 000a007e0c
work_peds_id 000a000f87
work_peds_id 000a007d1c
work_peds_id 000a009

In [118]:


def create_horse_results_table_():
    """
    horse_htmlを受け取って、結果テーブルに変換する関数。data/html/horse_resultsに保存する

    """

    horse_html_list = ['2017103612.bin','2015105131.bin']
    data_index = 1
    print("creating horse_results_table")
    horse_results = {}
    horse_id_df = pd.DataFrame(columns=["horse_id"])
    for horse_html in tqdm(horse_html_list):
        #horse_html_path = os.path.join('./data/tmp/horse', horse_html)
        #with open(horse_html_path, "rb") as f:
        try:
            # 保存してあるbinファイルを読み込む
            #html = f.read
            # パスから正規表現でhorse_id_listを取得
            #print("horse_html",horse_html)
            #print("horse_html_path",horse_html_path)

            horse_id_list = re.findall(r"(\d+).bin", horse_html)
            #print("horse_id_list",horse_id_list)
            # DataFrameに追加
            horse_id_df = pd.concat([horse_id_df, pd.DataFrame({"horse_id": horse_id_list})], ignore_index=True)
            print("horse_id_df ", horse_id_df)

            ### 取得日マスタの更新 ###
            print("updating master")
            now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # 現在日時を取得

            pd.DataFrame(columns=["horse_id", "updated_at"])
            # マスタを読み込み
            master = pd.read_csv("horse_results.pickle", dtype=object)
            # horse_id列に新しい馬を追加
            #new_master = master.merge(horse_id_df, on="horse_id", how="outer")
            # マスタ更新
            #new_master.loc[new_master["horse_id"].isin(horse_id_list), "updated_at"] = now
            # 列が入れ替わってしまう場合があるので、修正しつつ保存
            #target_data = new_master[["horse_id", "updated_at"]]
            #obtained_last_key = horse_html
        except Exception as e:
            print("Error at {}: {}".format(horse_html_path, e))

        data_index += 1


create_horse_results_table_()


creating horse_results_table


100%|██████████| 2/2 [00:00<00:00, 453.49it/s]

horse_id_df       horse_id
0  2017103612
updating master
horse_id_df       horse_id
0  2017103612
1  2015105131
updating master


In [19]:
def create_horse_results_table_():
    #
    # horseページのhtmlを受け取って、馬の過去成績のDataFrameに変換する関数。
    horse_html_list = ["2015110073.bin"]  # ,'2015105131.bin']
    data_index = 1
    horse_id_file_list = "./data/tmp/horse"

    print("scraping horse_results_table")
    horse_results = {}
    df = pd.DataFrame()
    for horse_html in tqdm(horse_html_list):
        horse_html_path = os.path.join("./data/html/horse", horse_html)
        with open(horse_html_path, "rb") as f:
            try:
                # 保存してあるbinファイルを読み込む
                html = f.read()
                # print("html",html)
                df = pd.read_html(html)[3]
                print("df", df)
                # 受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
                if df.columns[0] == "受賞歴":
                    df = pd.read_html(html)[4]
                    print("df4", df)
                # 新馬の競走馬レビューが付いた場合、
                # 列名に0が付与されるため、次のhtmlへ飛ばす
                if df.columns[0] == 0:
                    print("horse_results empty case1 {}".format(horse_html))
                    continue

                horse_id = re.findall(r"(\d+).bin", horse_html)[0]
                print("horse_id", horse_id)
                df.index = [horse_id] * len(df)
                horse_results[horse_id] = df

            # 競走データが無い場合（新馬）を飛ばす
            except IndexError:
                print("horse_results empty case2 {}".format(horse_html))
                continue

            except Exception as e:
                print("Error at {}: {}".format(horse_html, e))

        # pd.DataFrame型にして一つのデータにまとめる
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])
        # 列名に半角スペースがあれば除去する
    target_data = horse_results_df.rename(columns=lambda x: x.replace(" ", ""))
    print("target", target_data)


create_horse_results_table_()

scraping horse_results_table


100%|██████████| 1/1 [00:00<00:00, 30.72it/s]

df             日付     開催  天 気     R               レース名  映 像  頭 数  枠 番  馬 番  \
0   2023/12/24   5阪神8    晴   5.0          障害3歳以上未勝利  NaN   14  4.0    6   
1   2023/05/14   1京都8    曇  11.0             栗東S(L)  NaN   16  1.0    2   
2   2023/02/19   1東京8    晴  11.0        フェブラリーS(G1)  NaN   16  1.0    1   
3   2023/01/29   1東京2    晴  11.0            根岸S(G3)  NaN   16  6.0   11   
4   2022/12/11   5中山4    曇  11.0           カペラS(G3)  NaN   16  2.0    4   
5   2022/10/30   4東京9    晴  10.0         ペルセウスS(OP)  NaN   16  1.0    1   
6   2022/09/11   5中京2    晴  11.0      産経賞セントウルS(G2)  NaN   13  3.0    3   
7   2022/02/20   1阪神4    晴  11.0            大和S(OP)  NaN   16  1.0    2   
8   2022/01/30   1東京2    曇  11.0            根岸S(G3)  NaN   16  8.0   15   
9   2021/11/06   デルマー  NaN   NaN       BCダートマイル(G1)  NaN    8  NaN    4   
10  2021/09/11   5中京1    曇  11.0            エニフS(L)  NaN   16  6.0   11   
11  2021/05/01  2阪神11    曇  11.0           天王山S(OP)  NaN   15  1.0    1   
12  2021/04/18   3中山8 

In [88]:
def create_table_for_predict_():
    """
    scheduled_raceに保管したhtmlをスクレイピング。
    dateはyyyy/mm/ddの形式。
    """

    race_html_list = [
        "202410020110.bin",
        "202410020111.bin",
        "202405010511.bin",
        "202408020510.bin",
        "202410020109.bin",
        "202408020509.bin",
        "202405010510.bin",
        "202405010509.bin",
        "202408020511.bin",
    ]
    # print("race_html_list", race_html_list)

    df = pd.DataFrame()
    for race_html in tqdm(race_html_list):
        race_html_path = os.path.join("./data/html/scheduled_race/", race_html)
        with open(race_html_path, "rb") as f:
            # 保存してあるbinファイルを読み込む
            race_id = race_html[:12]
            html = f.read()
            df = pd.read_html(html)

            # htmlをsoupオブジェクトに変換
            soup = BeautifulSoup(html, "lxml")
            soup_span = soup.find_all("span")
            # メインのテーブルの取得
            for tr in soup.find_all("tr", class_="HorseList"):
                row = []
                for td in tr.find_all("td"):
                    if "HorseInfo" in td.get("class", []):
                        anchor = td.find("a")
                        if anchor:
                            href = anchor.get("href")
                            row.append(re.findall(r"horse/(\d*)", href)[0])
                    elif "Jockey" in td.get("class", []):
                        anchor = td.find("a")
                        if anchor:
                            href = anchor.get("href")
                            row.append(re.findall(r"jockey/result/recent/(\w*)", href)[0])
                    elif "Trainer" in td.get("class", []):
                        anchor = td.find("a")
                        if anchor:
                            href = anchor.get("href")
                            row.append(re.findall(r"trainer/result/recent/(\w*)", href)[0])
                    # elif "Txt_R Popular" in td.get("class", []):
                    #    span = td.find("span")
                    #    if span:
                    #        row.append(re.findall(r"(?<=>)\d+(\.\d+)?", span.text)[0])

                    row.append(td.text)
                print("row", row)
                # rowをDataFrameに変換し、dfに連結
                # DataFrameのリストを作成
                # 列のラベル
                columns = list(range(16))
                # DataFrameの作成
                df = pd.DataFrame([row], columns=columns)

                print("df", df)

                # レース結果テーブルと列を揃える
            df = df[[0, 1, 5, 6, 12, 13, 11, 3, 7, 9]]
            df.columns = [
                Cols.WAKUBAN,
                Cols.UMABAN,
                Cols.SEX_AGE,
                Cols.KINRYO,
                Cols.TANSHO_ODDS,
                Cols.POPULARITY,
                Cols.WEIGHT_AND_DIFF,
                "horse_id",
                "jockey_id",
                "trainer_id",
            ]
            df.index = [race_id] * len(df)

            print("df", df)


"""


            # レース情報の取得
            texts = soup.find_element(By.CLASS_NAME, "RaceList_Item02").text
            texts = re.findall(r"\w+", texts)
            # 障害レースフラグを初期化
            hurdle_race_flg = False
            for text in texts:
                if "m" in text:
                    # 20211212：[0]→[-1]に修正
                    df["course_len"] = [int(re.findall(r"\d+", text)[-1])] * len(df)
                if text in Master.WEATHER_LIST:
                    df["weather"] = [text] * len(df)
                if text in Master.GROUND_STATE_LIST:
                    df["ground_state"] = [text] * len(df)
                if "稍" in text:
                    df["ground_state"] = [Master.GROUND_STATE_LIST[1]] * len(df)
                if "不" in text:
                    df["ground_state"] = [Master.GROUND_STATE_LIST[3]] * len(df)
                if "芝" in text:
                    df["race_type"] = [list(Master.RACE_TYPE_DICT.values())[0]] * len(df)
                if "ダ" in text:
                    df["race_type"] = [list(Master.RACE_TYPE_DICT.values())[1]] * len(df)
                if "障" in text:
                    df["race_type"] = [list(Master.RACE_TYPE_DICT.values())[2]] * len(df)
                    hurdle_race_flg = True
                if "右" in text:
                    df["around"] = [Master.AROUND_LIST[0]] * len(df)
                if "左" in text:
                    df["around"] = [Master.AROUND_LIST[1]] * len(df)
                if "直線" in text:
                    df["around"] = [Master.AROUND_LIST[2]] * len(df)
                if "新馬" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[0]] * len(df)
                if "未勝利" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[1]] * len(df)
                if "１勝クラス" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[2]] * len(df)
                if "２勝クラス" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[3]] * len(df)
                if "３勝クラス" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[4]] * len(df)
                if "オープン" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[5]] * len(df)

            # グレードレース情報の取得
            if len(soup.find_elements(By.CLASS_NAME, "Icon_GradeType3")) > 0:
                df["race_class"] = [Master.RACE_CLASS_LIST[6]] * len(df)
            elif len(soup.find_elements(By.CLASS_NAME, "Icon_GradeType2")) > 0:
                df["race_class"] = [Master.RACE_CLASS_LIST[7]] * len(df)
            elif len(soup.find_elements(By.CLASS_NAME, "Icon_GradeType1")) > 0:
                df["race_class"] = [Master.RACE_CLASS_LIST[8]] * len(df)

            # 障害レースの場合
            if hurdle_race_flg:
                df["around"] = [Master.AROUND_LIST[3]] * len(df)
                df["race_class"] = [Master.RACE_CLASS_LIST[9]] * len(df)

            df["date"] = [date] * len(df)
        # except Exception as e:
        #    print(e)
        # finally:
        #    driver.close()
        #    driver.quit()
"""

create_table_for_predict_()

 22%|██▏       | 2/9 [00:00<00:00,  8.50it/s]

row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2018104534', '\n\n\nアスクキングコング\n\n\n', 'セ6', '55.0', '01174', '\n岩田望\n', '01084', '栗東梅田', '\n', '---.-', '\n**\n', '\n\n\n\n ', '']
df   0  1                        2           3                      4   5     6   \
0        \n\n--◎◯▲△☆&#10003消\n\n  2018104534  \n\n\nアスクキングコング\n\n\n  セ6  55.0   

      7        8      9     10  11     12      13         14 15  
0  01174  \n岩田望\n  01084  栗東梅田  \n  ---.-  \n**\n  \n\n\n\n      
row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2019104675', '\n\n\nアップストローク\n\n\n', 'セ5', '57.0', '01179', '\n菅原明\n', '01008', '美浦杉浦', '\n', '---.-', '\n**\n', '\n\n\n\n ', '']
df   0  1                        2           3                     4   5     6   \
0        \n\n--◎◯▲△☆&#10003消\n\n  2019104675  \n\n\nアップストローク\n\n\n  セ5  57.0   

      7        8      9     10  11     12      13         14 15  
0  01179  \n菅原明\n  01008  美浦杉浦  \n  ---.-  \n**\n  \n\n\n\n      
row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2020102971', '\n\n\nサト

 22%|██▏       | 2/9 [00:00<00:00,  7.26it/s]

row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2021105764', '\n\n\nアドマイヤベル\n\n\n', '牝3', '55.0', '\n\n○○\n\n', '01059', '美浦加藤征', '\n', '---.-', '\n**\n', '\n\n\n\n ', '']


ValueError: 16 columns passed, passed data had 15 columns

In [105]:
def load_file_pkl_():
    target_file_path = "./data/tmp/race_id_list/temp_race_id_list.txt"
    with open(target_file_path, "r") as f:
        try:
            lines = [line.strip() for line in f]
            # 探している数字を文字列として定義する
            target_number = "201006010106"

            # 文字列として比較する
            index = next((i for i, line in enumerate(lines) if line == target_number), None)

            if index is not None:
                if index < len(lines):
                    loaded_list = lines[:index+1]
                    return loaded_list
                else:
                    print("指定したIDがリストの最後にあります。")
            else:
                print("指定したIDがリスト内に見つかりません。")

        except ValueError:
            print("ファイルからの読み込み中にエラーが発生しました。")
            return None


# 関数を呼び出して実行する
result_list = load_file_pkl_()
print("リスト:", result_list)

リスト: ['201006010101', '201006010102', '201006010103', '201006010104', '201006010105', '201006010106']


In [34]:
from src.preparing.process_selector import select_process


def scrape_horse_id_list(df):
    """
    当日出走するhorse_id一覧を取得
    """
    ref_id_list = self.load_file_pkl()
    waiting_time = 10
    driver = prepare_chrome_driver()
    target_id_list = []
    # driver = prepare_chrome_driver()
    data_index = 1

    print("scraping horse_id_list")

    for ref_id in tqdm(ref_id_list):
        try:
            url = self.from_location + ref_id
            driver.get(url)
            wait = WebDriverWait(driver, waiting_time)
            wait.until(EC.presence_of_all_elements_located)
            html = urlopen(url)
            time.sleep(1)
            soup = BeautifulSoup(html, "lxml")
            # この例ではtarget=horse
            target_td_list = soup.find_all("td", attrs={"class": "txt_l"})
            target_ids = [a["href"] for td in target_td_list if (a := td.find("a")) and "/horse/" in a["href"]]
            target_id = [re.search(r"/horse/(\d+)/", href).group(1) for href in target_ids]
            target_id_list.append(target_id)

        except Exception as e:
            print("Error at {}: {}".format(ref_id, e))
            break
        if data_index % self.batch_size == 0:
            self.target_data = [item for sublist in target_id_list for item in sublist]
            self.save_temp_file("horse_id_list")
            self.obtained_last_key = target_id
            target_id_list = []
        data_index += 1
    # リストの要素を昇順にソート
    flat_list = [item for sublist in target_id_list for item in sublist]
    self.target_data = sorted(set(flat_list))
    self.save_temp_file("horse_id_list")
    self.obtained_last_key = target_id
    self.transfer_temp_file()


horse_id_list = select_process(alias="horse_id_list")

処理対象:horse_id_list
self.from_location:  https://db.netkeiba.com/race/
to_temp_location:  ./data/tmp/horse_id_list/
to_location:  ./data/html/horse_id_list/
self.from_local_location:  ./data/html/race_id_list/
self.from_local_file_name:  race_id_list.pkl
batch_size: 5
reloaded_race_id_list.pkl type:  race_id_list.pkl
scraping horse_id_list


  4%|▎         | 115/3168 [03:20<1:28:52,  1.75s/it]


KeyboardInterrupt: 

In [ ]:
def process_pkl_file(self, process_function):
    """
    binファイルを受け取って、テーブルに変換する関数。
    process_functionを入れ替えながら汎用的に使う共通モジュール
    対象ファイルや処理のバッチサイズなどを読み取り、セットの上、処理する
    """

    target_pkl_files = sorted(self.load_file_pkl(self))
    total_batches = (len(target_pkl_files) + self.batch_size - 1) // self.batch_size  # バッチ数の計算
    total_files = len(target_pkl_files)  # 処理対象の全データ数
    print(f"# of input files: {total_files}")
    processed_files = 0  # 処理済みのファイル数
    print(f"start {self.alias} processing")
    # target_data_name = {}
    driver = prepare_chrome_driver()
    # tqdmインスタンスの作成
    pbar = tqdm(total=total_files, desc="Processing Batches", unit="%", unit_scale=True, leave=True)

    for batch_index in range(total_batches):
        start_index = batch_index * self.batch_size
        end_index = min((batch_index + 1) * self.batch_size, len(target_pkl_files))
        batch_target_pkl_files = target_pkl_files[start_index:end_index]

        for ref_id in batch_target_pkl_files:  #
            try:
                temp_df = process_function(ref_id,driver)  # , target_data_name)

            except Exception as e:
                print("Error at {}: {}".format(ref_id, e))
                break

            processed_files += 1
            pbar.update(1)  # 処理済みのファイル数を1増やす
            temp_df = pd.concat([temp_df[key] for key in temp_df])

            # if self.alias == "race_results_table":
            #    self.target_data = trim_function(temp_df)
            self.target_data = temp_df

            self.save_temp_file(self.alias)
            # target_data_name = {}  # バッチ処理が完了したので辞書をクリア
            temp_df = []
        self.obtained_last_key = ref_id[-1]
        self.transfer_temp_file()
    print(f"# of processed files: {processed_files}")



def get_soup(self,ref_id,driver):
    url = self.from_location + ref_id
    driver.get(url)
    wait = WebDriverWait(driver, waiting_time=10)
    wait.until(EC.presence_of_all_elements_located)
    html = urlopen(url)
    time.sleep(1)
    soup = BeautifulSoup(html, "lxml")
    return soup

def get_raw_horse_id_list(ref_id,driver):
    soup = get_soup(ref_id, driver)
    # この例ではtarget=horse
    target_id_list=[]
    target_td_list = soup.find_all("td", attrs={"class": "txt_l"})
    target_ids = [a["href"] for td in target_td_list if (a := td.find("a")) and "/horse/" in a["href"]]
    target_id = [re.search(r"/horse/(\d+)/", href).group(1) for href in target_ids]
    for id in range(len(target_id)):
        target_id_list.append(target_id[id])
    target_id_list.append(target_id)
    return target_id_list

In [14]:
import pandas as pd
from abc import ABCMeta, abstractmethod


class Abstract:
    def __init__(self, filepath: str):
        self.__raw_data = pd.DataFrame(pd.read_pickle(filepath))
        self.__preprocessed_data = self._preprocess()

    @abstractmethod
    def _preprocess(self):
        pass

    @property
    def raw_data(self):
        new_columns = {col: int(col) if str(col).isdigit() else col for col in self.__raw_data.columns}
        # 列名を変換
        self.__raw_data.rename(columns=new_columns, inplace=True)
        return self.__raw_data.copy()

    @property
    def preprocessed_data(self):
        return self.__preprocessed_data.copy()


inst = Abstract("./data/raw/return_tables.pkl")
df = inst.raw_data
# new_columns = {col: int(col) if col.isdigit() else col for col in df.columns if col.isdigit()}
# 列名を変換
# df.rename(columns=new_columns, inplace=True)
print(df)

# 列名のリストを取得
column_names = df.columns

# 列名が整数かどうかをチェック
for name in column_names:
    if isinstance(name, int):
        print(f"列名 {name} は整数です。")
    else:
        print(f"列名 {name} は整数ではありません。")
# print(f"df['0']:{df['0']}")
print(f"df[0]:{df[0]}")

         0                        1                2         3       race_id
0       単勝                       14              240         1  202303010101
1       複勝                14br4br15    140br180br180   2br4br3  202303010101
2       枠連                    3 - 8              490         2  202303010101
3       馬連                   4 - 14              790         2  202303010101
4      ワイド  4 - 14br14 - 15br4 - 15    360br390br600   3br5br8  202303010101
...    ...                      ...              ...       ...           ...
11650   馬連                    2 - 3              940         2  202310020612
11651  ワイド    2 - 3br3 - 16br2 - 16  370br590br1,370  2br4br17  202310020612
11652   馬単                    3 → 2             1270         2  202310020612
11653  三連複               2 - 3 - 16             3700         8  202310020612
11654  三連単               3 → 2 → 16            11950        20  202310020612

[11655 rows x 5 columns]
列名 0 は整数です。
列名 1 は整数です。
列名 2 は整数です。
列名 3 は整数です。
列名

In [249]:
def create_raw_race_info_(target_bin_file_path):
    # print(f"target_bin_file_path:{target_bin_file_path}")
    with open(target_bin_file_path, "rb") as f:
        # 保存してあるbinファイルを読み込む
        html = f.read()

        # htmlをsoupオブジェクトに変換
        soup = BeautifulSoup(html, "lxml")
        # 天候、レースの種類、コースの長さ、馬場の状態、日付、回り、レースクラスをスクレイピング
        # 天候、レースの種類、コースの長さ、馬場の状態、日付、回り、レースクラスをスクレイピング
        texts = (
            soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
            + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
        )
        info = re.findall(r"\w+", texts)
        df = pd.DataFrame()
        # 障害レースフラグを初期化
        hurdle_race_flg = False
        for text in info:
            if text in ["芝", "ダート"]:
                df["race_type"] = [text]
            if "障" in text:
                df["race_type"] = ["障害"]
                hurdle_race_flg = True
            if "m" in text:
                # 20211212：[0]→[-1]に修正
                df["course_len"] = [int(re.findall(r"\d+", text)[-1])]
            if text in Master.GROUND_STATE_LIST:
                df["ground_state"] = [text]
            if text in Master.WEATHER_LIST:
                df["weather"] = [text]
            if "年" in text:
                df["date"] = [text]
            if "右" in text:
                df["around"] = [Master.AROUND_LIST[0]]
            if "左" in text:
                df["around"] = [Master.AROUND_LIST[1]]
            if "直線" in text:
                df["around"] = [Master.AROUND_LIST[2]]
            if "新馬" in text:
                df["race_class"] = [Master.RACE_CLASS_LIST[0]]
            if "未勝利" in text:
                df["race_class"] = [Master.RACE_CLASS_LIST[1]]
            if ("1勝クラス" in text) or ("500万下" in text):
                df["race_class"] = [Master.RACE_CLASS_LIST[2]]
            if ("2勝クラス" in text) or ("1000万下" in text):
                df["race_class"] = [Master.RACE_CLASS_LIST[3]]
            if ("3勝クラス" in text) or ("1600万下" in text):
                df["race_class"] = [Master.RACE_CLASS_LIST[4]]
            if "オープン" in text:
                df["race_class"] = [Master.RACE_CLASS_LIST[5]]

        # グレードレース情報の取得
        grade_text = soup.find("div", attrs={"class": "data_intro"}).find_all("h1")[0].text
        if "G3" in grade_text:
            df["race_class"] = [Master.RACE_CLASS_LIST[6]] * len(df)
        elif "G2" in grade_text:
            df["race_class"] = [Master.RACE_CLASS_LIST[7]] * len(df)
        elif "G1" in grade_text:
            df["race_class"] = [Master.RACE_CLASS_LIST[8]] * len(df)

        # 障害レースの場合
        if hurdle_race_flg:
            df["around"] = [Master.AROUND_LIST[3]]
            df["race_class"] = [Master.RACE_CLASS_LIST[9]]

        # インデックスをrace_idにする

        race_id = re.findall(r"race\W(\d+).bin", target_bin_file_path)[0]
        df.index = [race_id] * len(df)
        df[race_id] = df.index


create_raw_race_info_("./data/html/race/202006010101.bin")

FileNotFoundError: [Errno 2] No such file or directory: './data/html/race/202006010101.bin'

In [22]:
import re

# 仮のテキストとDataFrame
texts = [
    [
        "芝右",
        "外2400m",
        "天候",
        "曇",
        "芝",
        "良",
        "発走",
        "15",
        "35",
        "過去の日経新春杯",
        "2020年1月19日",
        "1回京都7日目",
        "4歳以上オープン",
        "国際",
        "特指",
        "ハンデ",
    ],
    [
        "ダ右1200m",
        "天候",
        "曇",
        "ダート",
        "良",
        "発走",
        "16",
        "10",
        "2020年1月19日",
        "1回京都7日目",
        "4歳以上2勝クラス",
        "指",
        "定量",
    ],
]
df_list = []

# テキストからコースの長さを抽出してDataFrameに設定する
for text_list in texts:
    course_len = None
    for text in text_list:
        match = re.search(r"\d{3,4}m", text)  # 正規表現でマッチング
        if match:
            course_len = int(match.group()[:-1])  # m を取り除いて整数に変換
            break  # コースの長さが見つかればループを終了
    df_list.append({"course_len": course_len})

print(df_list)

[{'course_len': 2400}, {'course_len': 1200}]


In [196]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [241]:
def dart_checker(text1):
    if text1.split("/")[0].strip()[:2] == "障芝":
        dart = True
    else:
        dart = False
    return dart


def count_ground_state(text1):
    return text1.split("/")[2].count(":")


def create_tmp_race_info_(target_bin_file_path):
    from src.constants._master import Master

    with open(target_bin_file_path, "rb") as f:
        # 保存してあるbinファイルを読み込む
        html = f.read()

        # htmlをsoupオブジェクトに変換
        soup = BeautifulSoup(html, "lxml")

        # 天候、レースの種類、コースの長さ、馬場の状態、日付、回り、レースクラスをスクレイピング
        text1 = soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
        text2 = soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
        # print(f"text1:{text1}")
        # print(f"text2:{text2}")
        race_id = re.findall(r"\d+", target_bin_file_path)[0]
        # テキスト情報を解析してDataFrameに変換
        race_distance = re.search(r"\d+", text1.split("/")[0]).group()
        weather = text1.split("/")[1].split(":")[1].strip()

        if weather in Master.WEATHER_LIST:
            pass
        else:
            print(f"unknown weather definition appeared:{race_id}")

        race_type = text1.split("/")[2].split(":")[0].strip()
        # 発走時刻
        start_time = text1.split("/")[-1].split(":")[1:3]
        start_time = ":".join(start_time).strip().split("\n\n")[0]

        race_date = text2.split(" ")[0]
        race_name = text2.split(" ")[1]
        dart = dart_checker(text1)
        print(f"dart:{dart}")
        print(f"count:{count_ground_state(text1)}")
        test = text1.split("/")[2].split(":")[2]
        print(f"gs2:{test}")

        for around in Master.AROUND_LIST:
            if around in text1.split("/")[0]:
                around_info = around
            else:
                around_info = None

        # 開催日数と開催回数を取得
        race_day_count = re.search(r"\d+", race_name.split("日目")[0]).group()  # 開催日数
        race_round_count = re.search(r"\d+", race_name.split("回")[0]).group()  # 開催回数

        # 開催場所を取得
        place_id = None
        for key, value in Master.PLACE_DICT.items():
            if key in race_name:
                place_id = value
                place_name = key
        if count_ground_state(text1) == 1:
            temp_ground_state0 = text1.split("/")[2].split(":")[1].strip()
            if temp_ground_state0 in Master.GROUND_STATE_LIST:
                ground_state1 = temp_ground_state0
                ground_state2 = temp_ground_state0
            else:
                print(f"unknown GROUND_STATE definition appeared1:{race_id}{ground_state1}{ground_state2}")
        elif dart_checker(text1) and count_ground_state(text1) == 2:
            temp_ground_state1 = text1.split("/")[2].split(":")[1].split()[0].strip()
            if temp_ground_state1 in Master.GROUND_STATE_LIST:
                ground_state1 = temp_ground_state1
            else:
                print(f"unknown GROUND_STATE definition appeared2:{race_id}{ground_state1}")
            temp_ground_state2 = text1.split("/")[2].split(":")[2].strip()
            if temp_ground_state2 in Master.GROUND_STATE_LIST:
                ground_state2 = temp_ground_state2
            else:
                print(f"unknown GROUND_STATE definition appeared3:{race_id}{temp_ground_state2}")
        # 不要な部分を削除
        # レース条件から年齢、性別、レースクラスを削除
        # 馬齢を取得
        race_condition = text2.split(" ")[2]

        # レース条件に基づいてフラグを設定
        race_flags = {}
        if race_condition is not None:
            # 性別を取得
            sex_info = None
            for sex in Master.SEX_LIST:
                if sex in race_condition:
                    sex_info = sex

            # レースクラスを取得
            race_class_info = None
            for race_class in Master.RACE_CLASS_LIST:
                if race_class in race_condition:
                    race_class_info = race_class
            if race_class_info is None:
                print(f"unknown race_class definition appeared:{race_id}")
            # 向きを取得

            if (around_info is None) and (("障害" in race_class_info or race_condition) or dart):
                around_info = "直線"

            for key, value in Master.RACE_CONDITION_DICT.items():
                if key in race_condition:
                    race_flags[value] = 1
                    race_condition = race_condition.replace(key, "").strip()
                else:
                    race_flags[value] = 0

        if "歳以上" in race_condition:
            age = re.search(r"\d+", race_condition.split("歳以上")[0]).group() + "+"

        else:
            age = re.search(r"\d+", race_condition.split("歳")[0]).group()
        if race_condition is not None:
            # ageの処理を修正
            if age is not None and age != "":
                if "+" in age:
                    race_condition = race_condition.replace(age[:-1], "").replace("歳以上", "").strip()
                else:
                    race_condition = race_condition.replace(age, "").replace("歳", "").strip()

            if sex_info is not None and sex_info != "":
                race_condition = race_condition.replace(sex_info, "").strip()
            if race_class_info is not None and race_class_info != "":
                race_condition = race_condition.replace(race_class_info, "").strip()
            if race_condition is not None and race_condition != "":
                race_condition = race_condition.replace("()", "").strip()
                race_condition = race_condition.replace("[]", "").strip()
            if race_condition is not None and race_condition != "":
                race_condition = race_condition.strip()

        # DataFrame作成歳
        df = pd.DataFrame(
            {
                # "レース名": [race_name],
                "レース場id": [place_id],
                "レース場名": [place_name],
                "開催日数": [race_day_count],  # 開催日数を追加
                "開催回数": [race_round_count],  # 開催回数を追加
                "レース開催日": [race_date],
                "発走時刻": [start_time],
                "レース種類": [race_type],
                "向き": [around_info],
                "レース距離": [race_distance],
                "天候": [weather],
                "馬場状態1": [ground_state1],
                "馬場状態2": [ground_state2],
                "馬齢": [age],
                "性別": [sex_info],
                "レースクラス": [race_class_info],
                "レース条件": [race_condition],
                **race_flags,
            }
        )
    return df


df = create_tmp_race_info_("./data/html/race/202305021104.bin")
print(df)

dart:True
count:2
gs2: 良 
  レース場id レース場名 開催日数 開催回数      レース開催日   発走時刻 レース種類  向き レース距離 天候  ... teiryo  \
0     05    東京    2    2  2023年5月27日  11:35     芝  直線  3000  晴  ...      1   

  barei bettei kokusai shimei tokushi  toku  handi  minarai  shogai  
0     0      0       0      0       0     0      0        0       1  

[1 rows x 26 columns]


In [37]:
def create_tmp_race_info_(target_bin_file_path):
    # print(f"target_bin_file_path:{target_bin_file_path}")
    with open(target_bin_file_path, "rb") as f:
        # 保存してあるbinファイルを読み込む
        html = f.read()
        # 馬の基本情報を取得
        #df = pd.read_html(html)[1].set_index(0).T
        # df = pd.read_html(html)
        # print(f"df:{df}")
        # htmlをsoupオブジェクトに変換
        soup = BeautifulSoup(html, "lxml")
        print(f"soup:{soup}")
        # 天候、レースの種類、コースの長さ、馬場の状態、日付、回り、レースクラスをスクレイピング
        #text1 = soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
        #text2 = soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
        ##print(f"text1:{text1}")
        #print(f"text2:{text2}")
        # info = re.findall(r"\w+", texts)
        # race_id = re.findall(r"\d+", target_bin_file_path)[0]
        # print(race_id, info)

    return


create_tmp_race_info_("./data/html/race/202303010101.bin")

soup:<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html id="html" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>3歳未勝利｜2023年4月8日 | 競馬データベース - netkeiba.com</title>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="ja" http-equiv="content-language"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="netkeiba.comが誇る国内最大級の競馬データベースです。50万頭以上の競走馬、騎手・調教師・馬主・生産者の全データがご覧いただけます。" name="description"/>
<meta content="競馬情報,競走馬,騎手,レース,調教師,馬主,検索,データベース,JRA,netkeiba,ネット競馬" name="keywords"/>
<!-- ogp用 -->
<meta content="netkeiba.com" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="競馬データベース | 競走馬・騎手など情報満載 - netkeiba" property="og:title"/>
<meta content="netkeiba.comが誇る国内最大級の競馬デ

In [46]:
from collections import Counter


def get_ids(df):
    # 空のDataFrameを作成
    ids_df = pd.DataFrame(columns=["race_id", "weather_id", "race_time_id", "race_date_id", "diff"])
    # 各race_idごとに処理を行う
    for race_id, group in df.groupby("race_id"):
        # "天候"が含まれている最初の行のidを記録する
        weather_id = group[group["info"].str.contains("天候")]["id"].iloc[0]
        # "発走"が含まれている最初の行のidを記録する
        race_time_id = group[group["info"].str.contains("発走")]["id"].iloc[0]
        race_date_id = group[group["info"].str.contains(r"\d{1,4}年\d{1,2}月\d{1,2}日")]["id"].iloc[0]
        # DataFrameに情報を追加
        ids_df = pd.concat(
            [
                ids_df,
                pd.DataFrame(
                    {
                        "race_id": [race_id],
                        "weather_id": [weather_id],
                        "race_time_id": [race_time_id],
                        "race_date_id": [race_date_id],
                        "diff": [race_time_id][0] - [weather_id][0],
                    }
                ),
            ]
        )

    return ids_df


def count_unique_values(df, column_name):
    """
    DataFrameの指定された列に含まれるユニークな値とその出現回数をカウントする関数。
    """
    # 指定された列のユニークな値を取得し、出現回数をカウントする
    unique_values_counts = df[column_name].value_counts()
    return unique_values_counts


df = pd.read_csv("./data/tmp/tmp_for_race_info/tmp_race_info_table.csv")
df = df.sort_values(by=["race_id", "id"])

print(f"df: {df.head(20)}")
columns = get_ids(df)
print(columns.tail())
print("max",df["id"].max())

# ユニークな値とその出現回数をカウントする列のリスト
columns_to_count = [
    "weather_id",
    "race_time_id",
    "race_date_id",
    "diff",

]  # 任意の列を追加可能

# 各列に対してユニークな値とその出現回数を取得し、表示する
for column_name in columns_to_count:
    print(f"{column_name}の内訳:")
    counts = count_unique_values(columns, column_name)
    print(counts)
    print()

# 4つの変数をzipして組み合わせを作成します
combined_variables = zip(
    columns["weather_id"], columns["race_time_id"], columns["race_date_id"]
)

# 組み合わせの数をカウントします
combination_counts = Counter(combined_variables)

# 結果を出力します
for combination, count in combination_counts.items():
    print(combination, ":", count)

df:     id       info       race_id
0    1    ダ右1700m  202303010101
1    2         天候  202303010101
2    3          曇  202303010101
3    4        ダート  202303010101
4    5          良  202303010101
5    6         発走  202303010101
6    7         09  202303010101
7    8         50  202303010101
8    9  2023年4月8日  202303010101
9   10    1回福島1日目  202303010101
10  11      3歳未勝利  202303010101
11  12          牝  202303010101
12  13          指  202303010101
13  14         馬齢  202303010101
14   1    芝右1200m  202303010102
15   2         天候  202303010102
16   3          曇  202303010102
17   4          芝  202303010102
18   5          良  202303010102
19   6         発走  202303010102
        race_id weather_id race_time_id race_date_id diff
0  202310020608          2            6            9    4
0  202310020609          2            6           10    4
0  202310020610          2            6           10    4
0  202310020611          2            6           10    4
0  202310020612          2        

In [67]:
df_info = pd.read_csv("./data/tmp/tmp_for_race_info/tmp_race_info_table.csv")
df_info = df_info.sort_values(by=["race_id", "id"])
df = pd.DataFrame()
print(f"df: {df_info.head(20)}")


# race_idをキーにして、"天候"を含む情報をリストにまとめる
def extract_weather_info(group):
    if any(group.iloc[2:5, :].eq("天候").any()):
        return [group.iloc[:, 1].values.tolist()]
    else:
        return None


# "天候"というinfoを持つ行のidを取得
weather_ids = df_info.loc[df_info["info"] == "天候", "id"]

# 対応するrace_idを取得
race_ids = df_info.loc[df_info["id"].isin(weather_ids), "race_id"]

# race_idをキーにして、対応するidを並べる
df = pd.DataFrame({"race_id": race_ids, "id": weather_ids}).groupby("race_id")["id"].apply(list).reset_index(name="ids")

print(df)


df:     id       info       race_id
0    1    ダ右1700m  202303010101
1    2         天候  202303010101
2    3          曇  202303010101
3    4        ダート  202303010101
4    5          良  202303010101
5    6         発走  202303010101
6    7         09  202303010101
7    8         50  202303010101
8    9  2023年4月8日  202303010101
9   10    1回福島1日目  202303010101
10  11      3歳未勝利  202303010101
11  12          牝  202303010101
12  13          指  202303010101
13  14         馬齢  202303010101
14   1    芝右1200m  202303010102
15   2         天候  202303010102
16   3          曇  202303010102
17   4          芝  202303010102
18   5          良  202303010102
19   6         発走  202303010102
           race_id              ids
0     202303010101  [2.0, nan, nan]
1     202303010102  [2.0, nan, nan]
2     202303010103  [2.0, nan, nan]
3     202303010104  [2.0, nan, nan]
4     202303010105  [2.0, nan, nan]
...            ...              ...
1459  202310020608  [2.0, nan, nan]
1460  202310020609  [2.0, nan, nan]


In [30]:
import pickle


def read_pkl_file(file_path):
    with open(file_path, "rb") as f:
        data = pickle.load(f)
    return data


# pklファイルのパスを指定してデータを読み込む
file_path = "./data/html/race_id_list/race_id_list.pkl"
pkl_data = read_pkl_file(file_path)
if pkl_data is not None:
    # 対象のデータがDataFrame内に存在するか確認し、位置を取得
    target_number = "202303010102"
    print(f"pkl_data:{pkl_data}")
    for index, value in pkl_data["race_id"].items():
        value_stripped = str(value).strip()
        if value_stripped == target_number:
            print(f"ここにあるよ: {index}")
            break
    else:
        print("対象のデータが見つかりませんでした。")
else:
    print("DataFrameがNoneを返しました。ファイルの読み込みに問題がある可能性があります。")

pkl_data:           race_id
0     202303010101
1     202303010102
2     202303010103
3     202303010104
4     202303010105
...            ...
1459  202310020608
1460  202310020609
1461  202310020610
1462  202310020611
1463  202310020612

[1464 rows x 1 columns]
ここにあるよ: 1


In [5]:
import pandas as pd
target_all_files = pd.date_range(start="2023-01-01", end="2023-05-02", freq="ME")
len(target_all_files)

4

In [6]:
import pandas as pd

# 例としてDataFrameを作成します
data = {"A": [1, 2, 3], "B": [4, 5, 6], "C": [7, 8, 9]}
df = pd.DataFrame(data)

# 列番号で昇順にソートします
sorted_columns = sorted(df.columns)
sorted_df = df[sorted_columns]

print(sorted_df)

   A  B  C
0  1  4  7
1  2  5  8
2  3  6  9


In [31]:
import pandas as pd

# サンプルのDataFrameを作成する
data = {"A": [1, 2, 3], "B": [4, 5, 6], "C": [7, 8, 9]}
df = pd.DataFrame(data)

# 最後の列を選択する
last_column = df.iloc[:, -1]
print(last_column)

0    7
1    8
2    9
Name: C, dtype: int64


In [53]:
import pandas as pd

# サンプルのDataFrame作成
data = {
    "race_id": [1, 1, 1, 2, 2, 2],
    "id": [1, 2, 3, 1, 2, 3],
    "col1": ["info1", "info2","info4", "info5", "info6"],
    "col2": ["天候", "info8", "info9","info11", "info12"],
    "col3": ["info13", "info15", "info16", "info17", "天候"],
}

df_info = pd.DataFrame(data)

# race_idをキーにして、"天候"を含む情報をリストにまとめる
df = (
    df_info.groupby("race_id")
    .apply(lambda x: x.iloc[:, 2:].values.tolist() if any(x.iloc[:, 2:5].eq("天候").any()) else None)
    .reset_index(name="天候")
)

print(df)

ValueError: All arrays must be of the same length

In [105]:
# import os
import pandas as pd
from numpy import NaN

# from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import re


# パターンにマッチする部分を抽出する関数を定義
def convert_string(value):
    # 正規表現パターンを定義
    pattern = r"\d{0,2}([^\d]+)\d{0,2}"
    match = re.search(pattern, value)
    if match:
        return match.group(1)  # マッチした部分の文字列を返す
    else:
        return value  # マッチしなかった場合は元の値を返す


def create_raw_horse_results_(target_bin_file_path):
    with open(target_bin_file_path, "rb") as f:
        # 保存してあるbinファイルを読み込む
        html = f.read()

        df = pd.read_html(html)[3]
        # 受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
        if df.columns[0] == "受賞歴":
            df = pd.read_html(html)[4]
            # print(f"test df:{df.iloc[:,1]}")

        # 新馬の競走馬レビューが付いた場合、
        # 列名に0が付与されるため、次のhtmlへ飛ばす
        if df.columns[0] == 0:
            print("horse_results empty case1 {}".format(target_bin_file_path))
            # continue

        # インデックスをhorse_idにする
        horse_id = re.findall(r"\d+", target_bin_file_path)[0]
        df.index = [horse_id] * len(df)
        df["horse_id"] = df.index

        # "R"列の値が数値を表す文字列であるかを判定し、数値を表す文字列の場合にintに変換する
        for index, value in df["R"].items():
            if isinstance(value, str) and value.isdigit():
                df.at[index, "R"] = int(value)

        # "R"列のデータ型をintに変換する
        df["R"] = pd.to_numeric(df["R"], errors="coerce").astype(float).astype("Int64")

        df.columns = df.columns.str.replace(" ", "")
        df.iloc[:, 1] = df.iloc[:, 1].apply(convert_string)

    return df


df = create_raw_horse_results_("./data/html/horse/2011106610.bin")
print(df.head())
print(df.dtypes)

                    日付  開催  天気   R       レース名  映像  頭数  枠番  馬番    オッズ  ...  \
2011106610  2024/02/07  高知   晴   4      C3ー10 NaN   9   7   7    1.8  ...   
2011106610  2024/01/31  高知  小雨   2       C3ー9 NaN   8   7   7    1.5  ...   
2011106610  2024/01/24  高知   晴   1      C3ー13 NaN   9   7   7    3.9  ...   
2011106610  2023/12/10  阪神   晴   8  3歳以上2勝クラス NaN   8   1   1  211.8  ...   
2011106610  2023/11/11  東京   曇  12  3歳以上2勝クラス NaN   8   7   7  206.9  ...   

            ﾀｲﾑ指数       通過        ペース    上り       馬体重 厩舎ｺﾒﾝﾄ  備考   勝ち馬(2着馬)  \
2011106610     **  5-6-4-4   0.0-39.7  40.0   479(-1)    NaN NaN      シシガオウ   
2011106610     **  4-1-2-2   0.0-40.5  41.9   480(-5)    NaN NaN   ナムラカメタロー   
2011106610     **  2-2-2-1   0.0-41.4  41.4   485(+5)    NaN NaN  (ララプロフォン)   
2011106610     **  8-8-8-7  38.2-33.7  36.4  480(-14)    NaN NaN   ウインルーティン   
2011106610     **      6-6  35.9-34.9  34.7  494(+16)    NaN NaN  ディオスバリエンテ   

              賞金    horse_id  
2011106610  12.0  2011106610  


In [26]:
# サンプルのDataFrameを作成
data = {"column_name": ["1京都1", "12阪神3", "6阪神13"]}
df = pd.DataFrame(data)


# パターンにマッチする部分を抽出する関数を定義
def convert_string(value):
    # 正規表現パターンを定義
    pattern = r"\d{0,2}([^\d]+)\d{0,2}"

    match = re.search(pattern, value)
    if match:
        return match.group(1)  # マッチした部分の文字列を返す
    else:
        return value  # マッチしなかった場合は元の値を返す


# 列の文字列を変換
df["column_name"] = df["column_name"].apply(convert_string)

print(df)

  column_name
0          京都
1          阪神
2          阪神


In [144]:
%autoreload

In [139]:
def trimmer_test(df):
    df["人気"] = df["人気"].fillna(0).astype(np.int64)
    return df


df = pd.read_csv("./data/tmp/race_results/temp_race_results_table.csv", dtype={"jockey_id":str, "trainer_id": str})
df = trimmer_test(df)

print(df.head())

  着順  枠番  馬番         馬名   性齢    斤量    騎手     タイム   着差    単勝  人気       馬体重  \
0  1   6   9      サペラヴィ   牡6  60.0  江田勇亮  3:08.7  NaN   6.5   2  474(-12)   
1  2   7  11  アストラエンブレム  セ10  60.0  五十嵐雄  3:09.2    3   9.5   5   490(-2)   
2  3   4   5   ゴールドフレーム   セ7  60.0   上野翔  3:09.3  1/2  18.7   8   482(-2)   
3  4   3   3    リーガルマナー   セ6  60.0  北沢伸也  3:09.8    3  33.8  12   494(-2)   
4  5   2   2  フロールシュタット   牡4  59.0  草野太郎  3:09.8   ハナ  19.0   9  478(+10)   

        調教師    horse_id jockey_id trainer_id  owner_id       race_id  
0  [東] 南田美知  2017100316     01023      00437    910006  202304010301  
1  [東] 小島茂之  2013106055     01069      01068    506800  202304010301  
2  [東] 新開幸一  2016105167     01087      01116    525030  202304010301  
3  [西] 四位洋文  2017105254     00691      01184    790005  202304010301  
4  [東] 戸田博文  2019105392     01113      01054    790005  202304010301  


In [4]:
import pandas as pd

from src.constants._results_cols import ResultsCols as Cols
from src.preprocessing._abstract_data_processor import AbstractDataProcessor


class ResultsProcessor(AbstractDataProcessor):
    def __init__(self, filepath):
        """
        初期処理
        """
        super().__init__(filepath)

    def _preprocess(self):
        """
        前処理
        """
        df = self.raw_data.copy()
        # print(f"type df:{type(df)}")

        # print(f"df.head:{df.head()}")
        # 着順の前処理
        df = self._preprocess_rank(df)

        # 性齢を性と年齢に分ける
        # サイト上のテーブルに存在する列名は、ResultsColsクラスで定数化している。
        df["性"] = df[Cols.SEX_AGE].map(lambda x: str(x)[0])
        df["年齢"] = df[Cols.SEX_AGE].map(lambda x: str(x)[1:]).astype(int)
        # print(f"type df:{type(df)}")

        # print(f"df.head:{df.head()}")

        # print(f"Cols.WEIGHT_AND_DIFF:{Cols.WEIGHT_AND_DIFF}")
        # print(f"Cols.WEIGHT_AND_DIFF.head():{df[Cols.WEIGHT_AND_DIFF].head()}")
        # 馬体重を体重と体重変化に分ける
        df["体重"] = df[Cols.WEIGHT_AND_DIFF].str.split("(", expand=True)[0]
        df["体重変化"] = df[Cols.WEIGHT_AND_DIFF].str.split("(", expand=True)[1].str[:-1]

        # errors='coerce'で、"計不"など変換できない時に欠損値にする
        df["体重"] = pd.to_numeric(df["体重"], errors="coerce")
        df["体重変化"] = pd.to_numeric(df["体重変化"], errors="coerce")

        # 各列を数値型に変換
        df[Cols.TANSHO_ODDS] = df[Cols.TANSHO_ODDS].astype(float)
        df[Cols.KINRYO] = df[Cols.KINRYO].astype(float)
        df[Cols.WAKUBAN] = df[Cols.WAKUBAN].astype(int)
        df[Cols.UMABAN] = df[Cols.UMABAN].astype(int)

        # 6/6出走数追加
        df["n_horses"] = df.index.map(df.index.value_counts())

        # カラム抽出
        df = self._select_columns(df)

        return df

    def _preprocess_rank(self, raw):
        """
        着順の前処理
        """
        df = raw.copy()

        # 着順に数字以外の文字列が含まれているものを取り除く

        df[Cols.RANK] = pd.to_numeric(df[Cols.RANK], errors="coerce")
        df.dropna(subset=[Cols.RANK], inplace=True)
        df[Cols.RANK] = df[Cols.RANK].astype(int)
        df["rank"] = df[Cols.RANK].map(lambda x: 1 if x < 4 else 0)
        return df

    def _select_columns(self, raw):
        """
        カラム抽出
        """
        df = raw.copy()[
            [
                # Cols.RANK, # 着順
                Cols.WAKUBAN,  # 枠番
                Cols.UMABAN,  # 馬番
                # Cols.HORSE_NAME, # 馬名
                # Cols.SEX_AGE, # 性齢
                Cols.KINRYO,  # 斤量
                # Cols.JOCKEY, # 騎手
                # Cols.TIME # タイム
                # Cols.RANK_DIFF # 着差
                Cols.TANSHO_ODDS,  # 単勝
                # Cols.POPULARITY, # 人気
                # Cols.WEIGHT_AND_DIFF, # 馬体重
                # Cols.TRAINER, # 調教師
                "race_id",
                "horse_id",
                "jockey_id",
                "trainer_id",
                "owner_id",
                "性",
                "年齢",
                "体重",
                "体重変化",
                "n_horses",
                "rank",
            ]
        ]
        return df


inst = ResultsProcessor("./data/raw/results.pkl")
df_copy = inst.raw_data.copy()
df = inst._preprocess()

print(df_copy.head())
print(df_copy.dtypes)
print(df.index)
# df["race_id"].isna().sum()
df["性"].value_counts()

  着順  枠番  馬番         馬名  性齢    斤量    騎手     タイム     着差    単勝   人気       馬体重  \
0  1   5   5    サトミノキラリ  牡2  55.0  横山武史  1:09.5    NaN   1.2  1.0   452(-4)   
1  2   8   8     ベアゴーゴー  牝2  55.0   浜中俊  1:09.5     クビ   4.1  2.0   454(+2)   
2  3   6   6  ハピアーザンエバー  牡2  55.0  藤岡佑介  1:10.0  2.1/2  59.9  6.0   438(-6)   
3  4   4   4  デビルシズカチャン  牝2  55.0  ルメール  1:10.2  1.1/2  16.6  3.0   450(+2)   
4  5   1   1   ウィスピースノー  牝2  55.0  吉田隼人  1:10.3    1/2  23.9  5.0  434(-10)   

        調教師    horse_id  jockey_id  trainer_id  owner_id       race_id  
0  [東] 鈴木伸尋  2021101429       1170        1026    425031  202301010101  
1  [東] 杉浦宏昭  2021105872       1115        1008    372033  202301010101  
2  [西] 羽月友彦  2021106854       1093        1091    464030  202301010101  
3  [西] 武幸四郎  2021105553       5339        1160    680031  202301010101  
4  [西] 今野貞一  2021100648       1095        1128    281008  202301010101  
着順             object
枠番              int64
馬番              int64
馬名             object

性
牡    27310
牝    21535
セ     2484
Name: count, dtype: int64

In [2]:
import pandas as pd

from src.preprocessing._abstract_data_processor import AbstractDataProcessor


class RaceInfoProcessor(AbstractDataProcessor):
    def __init__(self, filepath):
        """
        初期処理
        """
        super().__init__(filepath)

    def _preprocess(self):
        """
        前処理
        """
        df = self.raw_data
        # 距離は10の位を切り捨てる
        df["course_len"] = df["course_len"].astype(float) // 100
        # 日付型に変更
        df["date"] = pd.to_datetime(df["date"], format="%Y年%m月%d日")
        # 開催場所

        return df


inst = RaceInfoProcessor("./data/raw/race_info.pkl")
df = inst._preprocess()
print(df.head())
print(df.dtypes)
print(df.index)
print(df["place_id"].isna().sum())
df["sex"].value_counts()

        race_id  place_id place  days  times       date   time race_type  \
0  202301010101         1    札幌     1      1 2023-07-22  09:50         芝   
1  202301010102         1    札幌     1      1 2023-07-22  10:25       ダート   
2  202301010103         1    札幌     1      1 2023-07-22  10:55       ダート   
3  202301010104         1    札幌     1      1 2023-07-22  11:25         芝   
4  202301010105         1    札幌     1      1 2023-07-22  12:15       ダート   

  around  course_len  ... tokushi toku handi minarai wakate shogai chiho  gai  \
0     直線        12.0  ...       0    0     0       0      0      0     0    0   
1     直線        10.0  ...       0    0     0       0      0      0     0    0   
2     直線        17.0  ...       0    0     0       0      0      0     0    0   
3     直線        15.0  ...       0    0     0       0      0      0     0    0   
4     直線        17.0  ...       0    0     0       0      0      0     0    0   

   kongo  hinba  
0      1      0  
1      0      0  
2 

sex
混    1899
牝     496
Name: count, dtype: int64

In [124]:
import pandas as pd

from src.preprocessing._abstract_data_processor import AbstractDataProcessor


class ReturnProcessor(AbstractDataProcessor):
    def __init__(self, filepath):
        """
        初期処理
        """
        super().__init__(filepath)

    def _preprocess(self):
        """
        前処理
        """

        return_dict = {}
        return_dict["tansho"] = self.__tansho().fillna(0).astype(int)
        return_dict["fukusho"] = self.__fukusho().fillna(0).astype(int)
        return_dict["umaren"] = self.__umaren().fillna(0).astype(int)
        return_dict["umatan"] = self.__umatan().fillna(0).astype(int)
        return_dict["wide"] = self.__wide().fillna(0).astype(int)
        return_dict["sanrentan"] = self.__sanrentan().fillna(0).astype(int)
        return_dict["sanrenpuku"] = self.__sanrenpuku().fillna(0).astype(int)
        return return_dict

    def __tansho(self):
        """
        単勝
        """
        tansho = self.raw_data[self.raw_data[0] == "単勝"][[1, 2]]
        tansho.columns = ["win", "return"]

        for column in tansho.columns:
            tansho[column] = pd.to_numeric(tansho[column], errors="coerce")

        return tansho

    def __fukusho(self):
        """
        複勝
        """
        fukusho = self.raw_data[self.raw_data[0] == "複勝"][[1, 2]]
        wins = fukusho[1].str.split("br", expand=True)[[0, 1, 2]]

        wins.columns = ["win_0", "win_1", "win_2"]
        returns = fukusho[2].str.split("br", expand=True)[[0, 1, 2]]
        returns.columns = ["return_0", "return_1", "return_2"]

        df = pd.concat([wins, returns], axis=1)
        for column in df.columns:
            df[column] = df[column].str.replace(",", "")
        return df.fillna(0).astype(int)

    def __umaren(self):
        """
        馬連
        """
        umaren = self.raw_data[self.raw_data[0] == "馬連"][[1, 2]]
        wins = umaren[1].str.split("-", expand=True)[[0, 1]].add_prefix("win_")
        return_ = umaren[2].rename("return")
        df = pd.concat([wins, return_], axis=1)
        return df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    def __umatan(self):
        """
        馬単
        """
        umatan = self.raw_data[self.raw_data[0] == "馬単"][[1, 2]]
        wins = umatan[1].str.split("→", expand=True)[[0, 1]].add_prefix("win_")
        return_ = umatan[2].rename("return")
        df = pd.concat([wins, return_], axis=1)
        return df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    def __wide(self):
        """
        ワイド
        """
        wide = self.raw_data[self.raw_data[0] == "ワイド"][[1, 2]]
        wins = wide[1].str.split("br", expand=True)[[0, 1, 2]]
        wins = wins.stack().str.split("-", expand=True).add_prefix("win_")
        return_ = wide[2].str.split("br", expand=True)[[0, 1, 2]]
        return_ = return_.stack().rename("return")
        df = pd.concat([wins, return_], axis=1)
        return df.apply(lambda x: pd.to_numeric(x.str.replace(",", ""), errors="coerce"))

    def __sanrentan(self):
        """
        三連単
        """
        rentan = self.raw_data[self.raw_data[0] == "三連単"][[1, 2]]
        wins = rentan[1].str.split("→", expand=True)[[0, 1, 2]].add_prefix("win_")
        return_ = rentan[2].rename("return")
        df = pd.concat([wins, return_], axis=1)
        return df.apply(lambda x: pd.to_numeric(x, errors="coerce"))

    def __sanrenpuku(self):
        """
        三連複
        """
        renpuku = self.raw_data[self.raw_data[0] == "三連複"][[1, 2]]
        wins = renpuku[1].str.split("-", expand=True)[[0, 1, 2]].add_prefix("win_")
        return_ = renpuku[2].rename("return")
        df = pd.concat([wins, return_], axis=1)
        return df.apply(lambda x: pd.to_numeric(x, errors="coerce"))


inst = ReturnProcessor("./data/raw/return_tables.pkl")
df = inst._preprocess()
print(df)
print(df.index)

{'tansho':       win  return
0       9     650
8      13     520
16      6     400
24      6     310
31      3     400
...   ...     ...
2061    1     200
2069   13     670
2077   12     310
2085    8     360
2092    8     910

[264 rows x 2 columns], 'fukusho':       win_0  win_1  win_2  return_0  return_1  return_2
1         9     11      5       230       320       570
9        13     15     14       170       480       190
17        6     10      7       170       350       230
25        6      2      4       150       320       160
32        3     10      4       160       190       190
...     ...    ...    ...       ...       ...       ...
2062      1     13     16       130      1180       640
2070     13      9     15       180      1000       180
2078     12     15      7       150       160       330
2086      8      7      1       140       180       150
2093      8     12      2       280       250       390

[264 rows x 6 columns], 'umaren':       win_0  win_1  return
3  

AttributeError: 'dict' object has no attribute 'index'

In [125]:
import pandas as pd

from src.constants._horse_info_cols import HorseInfoCols as Cols
from src.preprocessing._abstract_data_processor import AbstractDataProcessor


class HorseInfoProcessor(AbstractDataProcessor):
    def __init__(self, filepath):
        """
        初期処理
        """
        super().__init__(filepath)

    def _preprocess(self):
        """
        前処理
        """
        df = self.raw_data

        # 生年月日をdatetime型に設定
        df["birthday"] = pd.to_datetime(df[Cols.BIRTHDAY], format="%Y年%m月%d日")

        # インデックス名を与える
        df["horse_id"].astype(int)
        df.set_index("horse_id", inplace=True)
        # カラム抽出
        df = self._select_columns(df)

        return df

    def _select_columns(self, raw):
        """
        カラム抽出
        """
        df = raw.copy()[
            [
                # Cols.BIRTHDAY, # 生年月日
                # Cols.TRAINER, # 調教師
                # Cols.OWNER, # 馬主
                # Cols.BREEDER, # 生産者
                # Cols.REC_INFO, # 募集情報
                # Cols.ORIGIN, # 産地
                # Cols.PRICE, # セリ取引価格
                # Cols.WINNING_PRIZE, # 獲得賞金
                # Cols.TOTAL_RESULTS, # 通算成績
                # Cols.VICTORY_RACE, # 人気
                # Cols.RELATIVE_HORSE, # 近親馬
                "birthday",
                #'trainer_id',
                "owner_id",
                "breeder_id",
            ]
        ]

        return df


inst = HorseInfoProcessor("./data/raw/horse_info.pkl")
df = inst._preprocess()
print(df.head())
print(df.dtypes)
print(df.index)

             birthday owner_id breeder_id
horse_id                                 
2011106610 2011-05-29   851009     233302
2013101904 2013-05-12   546800     710303
2013106055 2013-04-06   506800     373126
2014100063 2014-02-24   812008     830333
2014100555 2014-04-06   329004     800314
birthday      datetime64[ns]
owner_id              object
breeder_id            object
dtype: object
Index([2011106610, 2013101904, 2013106055, 2014100063, 2014100555, 2014100566,
       2014102562, 2014103266, 2014103547, 2014103963,
       ...
       2020110121, 2020110122, 2020110128, 2020110130, 2020110133, 2020110138,
       2020110140, 2020110142, 2020110149, 2020110151],
      dtype='int64', name='horse_id', length=3261)


In [208]:
import re

import pandas as pd

from src.constants._horse_results_cols import HorseResultsCols as Cols
from src.constants._master import Master
from src.preprocessing._abstract_data_processor import AbstractDataProcessor


class HorseResultsProcessor(AbstractDataProcessor):
    def __init__(self, filepath):
        """
        初期処理
        """
        super().__init__(filepath)

    def _preprocess(self):
        """
        前処理
        """
        df = self.raw_data

        # 着順に数字以外の文字列が含まれているものは、欠損値（NaN）に置き換える
        # サイト上のテーブルに存在する列名は、HorseResultsColsクラスで定数化している。
        df[Cols.RANK] = pd.to_numeric(df[Cols.RANK], errors="coerce")
        # 着順が欠損値（NaN）となったものを取り除く
        df.dropna(subset=[Cols.RANK], inplace=True)
        df[Cols.RANK] = df[Cols.RANK].astype(int)

        # 日付をdatetime型に設定
        df["date"] = pd.to_datetime(df[Cols.DATE])

        # 賞金のNaNを0で埋める
        df[Cols.PRIZE] = df[Cols.PRIZE].fillna(0)
        # df[{Cols.PRIZE:0}].fillna(inplace=True)
        # df[Cols.PRIZE].fillna(0, inplace=True) で怒られたが、df[Cols.PRIZE].fillna(0, inplace=True)　も怒られた
        #'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead

        # 1着の着差を0にする（xが0より小さい場合は、0、xが0以上の場合、xを返す）
        df[Cols.RANK_DIFF] = df[Cols.RANK_DIFF].map(lambda x: 0 if x < 0 else x)

        # レース展開データ
        # n=1: 最初のコーナー位置, n=4: 最終コーナー位置
        def corner(x, n):
            if not isinstance(x, str):
                return x
            elif n == 4:
                return int(re.findall(r"\d+", x)[-1])
            elif n == 1:
                return int(re.findall(r"\d+", x)[0])

        df["first_corner"] = df[Cols.CORNER].map(lambda x: corner(x, 1))
        df["final_corner"] = df[Cols.CORNER].map(lambda x: corner(x, 4))

        df["final_to_rank"] = df["final_corner"] - df[Cols.RANK]
        df["first_to_rank"] = df["first_corner"] - df[Cols.RANK]
        df["first_to_final"] = df["first_corner"] - df["final_corner"]

        # 開催場所（数字以外の文字列を抽出）中央開催・地方開催・海外開催以外をその他（'99'）とする
        df[Cols.PLACE] = df[Cols.PLACE].str.extract(r"(\D+)")[0].map(Master.PLACE_DICT).fillna("99")

        # race_type（数字以外の文字列を抽出）
        df["race_type"] = df[Cols.RACE_TYPE_COURSE_LEN].str.extract(r"(\D+)")[0].map(Master.RACE_TYPE_DICT)
        # 距離は10の位を切り捨てる（数字の文字列を抽出）
        df["course_len"] = df[Cols.RACE_TYPE_COURSE_LEN].str.extract(r"(\d+)").astype(float) // 100

        # タイムの値を秒単位に変換
        # 準備
        baseformat = "%M:%S.%f"
        basetime = pd.to_datetime("00:00.0", format=baseformat)
        to_datetime = lambda x: pd.to_datetime(df[Cols.TIME], format=x, errors="coerce")
        # 秒単位へのフォーマット変換処理
        datetime_s = to_datetime(baseformat)
        # 「x:xx.x」フォーマット以外、許容するフォーマットを定義
        formats_additional = ["%M.%S.%f", "%M:%S:%f"]
        for format_ in formats_additional:
            # 秒単位へのフォーマット変換処理
            datetime_s = datetime_s.fillna(to_datetime(format_))
        # フォーマット例外は欠損値になる
        df["time_seconds"] = (datetime_s - basetime).dt.total_seconds()

        # インデックス名を与える
        # df.index.name = "horse_id"
        df["horse_id"].astype(int)
        df.set_index("horse_id", inplace=True)

        # カラム抽出
        df = self._select_columns(df)

        return df

    def _select_columns(self, raw):
        """
        カラム抽出
        """
        df = raw.copy()[
            [
                # Cols.DATE, # 日付
                Cols.PLACE,  # 開催
                Cols.WEATHER,  # 天気
                Cols.R,  # R
                Cols.RACE_NAME,  # レース名
                # 映像
                Cols.N_HORSES,  # 頭数
                Cols.WAKUBAN,  # 枠番
                Cols.UMABAN,  # 馬番
                Cols.TANSHO_ODDS,  # オッズ
                Cols.POPULARITY,  # 人気
                Cols.RANK,  # 着順
                Cols.JOCKEY,  # 騎手
                Cols.KINRYO,  # 斤量
                # Cols.RACE_TYPE_COURSE_LEN, # 距離
                Cols.GROUND_STATE,  # 馬場
                # 馬場指数
                # Cols.TIME, # タイム
                Cols.RANK_DIFF,  # 着差
                # ﾀｲﾑ指数
                Cols.CORNER,  # 通過
                Cols.PACE,  # ペース
                Cols.NOBORI,  # 上り
                Cols.WEIGHT_AND_DIFF,  # 馬体重
                # 厩舎ｺﾒﾝﾄ
                # 備考
                # 勝ち馬(2着馬)
                Cols.PRIZE,  # 賞金
                "date",
                "first_corner",
                "final_corner",
                "final_to_rank",
                "first_to_rank",
                "first_to_final",
                "race_type",
                "course_len",
                "time_seconds",
            ]
        ]

        return df


inst = HorseResultsProcessor("./data/html/horse_results/horse_results.pkl")
# df_copy = inst.raw_data.copy()
df = inst._preprocess()

print(df.dtypes)
print(df.head(1))
# print(df_copy["枠番"].isna().sum())
# print(df_copy[df_copy["枠番"].isna()])
# print(df.index)
#print(df["開催"].isna().sum())

開催                        object
天気                        object
R                        float64
レース名                      object
頭数                         int64
枠番                       float64
馬番                         int64
オッズ                      float64
人気                       float64
着順                         int64
騎手                        object
斤量                       float64
馬場                        object
着差                       float64
通過                        object
ペース                       object
上り                       float64
馬体重                       object
賞金                       float64
date              datetime64[ns]
first_corner             float64
final_corner             float64
final_to_rank            float64
first_to_rank            float64
first_to_final           float64
race_type                 object
course_len               float64
time_seconds             float64
dtype: object
            開催 天気    R   レース名  頭数   枠番  馬番  オッズ   人気  着順  ... 

In [250]:
from sklearn.preprocessing import LabelEncoder

from src.preprocessing._abstract_data_processor import AbstractDataProcessor


class PedsProcessor(AbstractDataProcessor):
    """
    初期処理
    """

    def __init__(self, filepath):
        super().__init__(filepath)

    """
    前処理
    """

    def _preprocess(self):
        df = self.raw_data.copy()
        df["horse_id"] = df["horse_id"].astype(int)  # horse_id 列を整数型に変換

        # horse_id 列を除く列をカテゴリ型に変換する
        for column in df.columns[:-1]:
            if column != "horse_id":
                df[column] = LabelEncoder().fit_transform(df[column].fillna("Na"))
                df[column] = df[column].astype("category")
        # horse_id 列をインデックスに設定する
        df.set_index("horse_id", inplace=True)
        return df


inst = PedsProcessor("./data/raw/peds.pkl")
df = inst._preprocess()
print(df.head())
print(df.dtypes)
print(df.index)

           peds_0 peds_1 peds_2 peds_3 peds_4 peds_5 peds_6 peds_7 peds_8  \
horse_id                                                                    
2011106610    128      1     18     14     19      9     10     25     20   
2013101904    114      9     17     14     19      9      7     44     11   
2013106055    139      9     17     14     19      9      7     44     11   
2014100063    143      9     17     14     19      9      7     44     11   
2014100555    143      9     17     14     19      9      7     44     11   

           peds_9  ... peds_52 peds_53 peds_54 peds_55 peds_56 peds_57  \
horse_id           ...                                                   
2011106610     25  ...     244      74     295     673     229      86   
2013101904     31  ...      34     106      38     101     226     142   
2013106055     31  ...     282     205     330     107     208      52   
2014100063     31  ...     137     160     178     118     264     190   
2014100555     3

In [241]:
%autoreload

In [243]:
import pandas as pd
from tqdm.auto import tqdm

from src.preprocessing._horse_info_processor import HorseInfoProcessor
from src.preprocessing._horse_results_processor import HorseResultsProcessor
from src.preprocessing._peds_processor import PedsProcessor
from src.preprocessing._race_info_processor import RaceInfoProcessor
from src.preprocessing._results_processor import ResultsProcessor
from src.constants._horse_results_cols import HorseResultsCols
from src.constants._results_cols import ResultsCols
import sys

# 表示を省略しないように設定
sys.maxsize = sys.maxsize

"""
class DataMerger:
    def __init__(
        self,
        results_processor: ResultsProcessor,
        race_info_processor: RaceInfoProcessor,
        horse_results_processor: HorseResultsProcessor,
        horse_info_processor: HorseInfoProcessor,
        peds_processor: PedsProcessor,
        target_cols: list,
        group_cols: list,
    ):
"""


# レース結果テーブル（前処理後）
# _results = results_processor.preprocessed_data
# レース情報テーブル（前処理後）
# _race_info = race_info_processor.preprocessed_data
# 馬の過去成績テーブル（前処理後）
# _horse_results = horse_results_processor.preprocessed_data
# 馬の基本情報テーブル（前処理後）
# 馬主情報はレース情報テーブルのものを利用するため、列を削除
# _horse_info = horse_info_processor.preprocessed_data.drop(["owner_id"], axis=1)
# 血統テーブル（前処理後）
# _peds = peds_processor.preprocessed_data
# 集計対象列
# ターゲットエンコーディング時に「馬の成績」として扱う項目
TARGET_COLS = [
    HorseResultsCols.RANK,
    HorseResultsCols.PRIZE,
    HorseResultsCols.RANK_DIFF,
    "first_corner",
    "final_corner",
    "first_to_rank",
    "first_to_final",
    "final_to_rank",
    "time_seconds",
]
# horse_id列と共に、ターゲットエンコーディングの対象にする列
GROUP_COLS = ["course_len", "race_type", HorseResultsCols.PLACE]

_target_cols = TARGET_COLS
# horse_idと一緒にターゲットエンコーディングしたいカテゴリ変数
_group_cols = GROUP_COLS
# 全てのマージが完了したデータ
_merged_data = pd.DataFrame()
# 日付(date列)ごとに分かれたレース結果
_separated_results_dict = {}
# レース結果データのdateごとに分かれた馬の過去成績
_separated_horse_results_dict = {}


def convert_to_datetime(date_str, time_str):
    # 日付と時刻を結合して datetime オブジェクトに変換
    datetime_str = date_str.split()[0] + " " + time_str
    return datetime_str


def convert_column_types(df, column_types_dict):
    """
        DataFrameの指定された列の型を変換する関数。

        Parameters:
        df (DataFrame): 変換対象のDataFrame。
        column_types (dict): 列名と変換後の型のペアを持つ辞書。

        Returns:
        DataFrame: 型を変換したDataFrame。

    {"place_id": ("str","str", 2, True),
      "race_id": ("float","int", 2, False) ,
      "trainer_id": ("int","str", 5, True) ,
      "owner_id": ("int","str", 6, True)}
    """
    converted_df = df.copy()
    for column, (from_data_type, to_data_type, width, padding) in column_types_dict.items():
        if from_data_type == "str":
            if to_data_type == "str":
                if padding:
                    # ゼロパディングが必要な場合
                    converted_df[column] = converted_df[column].astype(str).str.zfill(width)
                else:
                    # ゼロパディングが不要な場合
                    converted_df[column] = converted_df[column].astype(to_data_type)

        if from_data_type == "int":
            if to_data_type == "str":
                if padding:
                    # ゼロパディングが必要な場合
                    converted_df[column] = converted_df[column].astype(str).str.zfill(width)
                else:
                    # ゼロパディングが不要な場合
                    converted_df[column] = converted_df[column].astype(to_data_type)

    for column, (from_data_type, to_data_type, width, padding) in column_types_dict.items():
        if from_data_type == "float":
            if to_data_type == "str":
                if padding:
                    # ゼロパディングが必要な場合
                    converted_df[column] = (
                        converted_df[column].astype(float).astype(str).str.split(".").str[0].str.zfill(width)
                    )
                else:
                    # ゼロパディングが不要な場合
                    converted_df[column] = converted_df[column].astype(to_data_type)
            if to_data_type == "int":
                converted_df[column] = converted_df[column].fillna(0).astype(to_data_type)

    return converted_df


def _filter_horse_results(horse_results, n_races):
    """
    直近nレースに絞る
    """
    return horse_results.sort_values("date", ascending=False).groupby(level=0).head(n_races)


def _sort_horse_results(horse_results):
    """
    全レース対象にしてソート
    """
    return horse_results.sort_values("date", ascending=False).groupby(level=0)


def _summarize(horse_results, target_cols):
    """
    horse_idごとに、target_colsを集計する
    """
    return horse_results.groupby(level=0)[target_cols].mean()


def _summarize_with(horse_results, target_cols, group_col):
    """
    horse_id, group_colごとにtarget_colsを集計する
    """
    return horse_results.groupby(["horse_id", group_col])[target_cols].mean()


def merge():
    """
    マージ処理
    """
    _merge_race_info()
    print("merge_race_info() Done")
    _merge_horse_results()
    print("merge_horse_results() Done")
    _merge_horse_info()
    _merge_peds()


def _merge_race_info(_results, _race_info):
    """
    レース情報テーブルを、レース結果テーブルにマージ
    """
    _results = _results.merge(_race_info, left_index=True, right_index=True, how="left")
    return _results


def _separate_by_date(_results, _horse_results):
    # global _separated_results_dict
    # global _separated_horse_results_dict
    """
    - レース結果を日付(date列)ごとに分ける
    - (レース結果のdate, その日に走る馬の過去成績データ)のペアを作る
    """
    print("separating horse results by date")
    # dateでデータを分割
    for date, df_by_date in tqdm(_results.groupby("date")):
        _separated_results_dict[date] = df_by_date
        # その日に走る馬一覧
        horse_id_list = df_by_date["horse_id"].unique()
        # dateより過去に絞る
        _separated_horse_results_dict[date] = _horse_results.query("date < @date").query("horse_id in @horse_id_list")


def _merge_horse_results(_results, _horse_results):
    """
    馬の過去成績テーブルのマージ
    """
    n_races_list = [435, 440]
    _separate_by_date(_results, _horse_results)
    print("merging horse_results")
    output_results_dict = {}
    for date in tqdm(_separated_results_dict):
        _results = _separated_results_dict[date].copy()
        _horse_results = _separated_horse_results_dict[date].copy()

        # 直近nレースに絞った過去成績をマージ
        for n_races in n_races_list:
            # 直近nレースに絞った過去成績
            n_race_horse_results = _filter_horse_results(_horse_results, n_races)
            # print("n_race_horse_results:",n_race_horse_results)

            # horse_idのみのターゲットエンコーディング
            # 何レース分集計しているか分かるように、列名に接尾辞をつける
            _summarized = _summarize(n_race_horse_results, _target_cols).add_suffix("_{}R".format(n_races))
            # print(f"summarized:{summarized}")
            # resultsにマージ
            _results = _results.merge(_summarized, left_on="horse_id", right_index=True, how="left")
            #print("sp_results_after 1st merge", sp_results.T)
            # horse_idとカテゴリ変数を合わせてターゲットエンコーディング
            for group_col in _group_cols:
                # 何レース分、どのカテゴリ変数とともに集計しているか分かるように、列名に接尾辞をつける
                _summarized_with = _summarize_with(n_race_horse_results, _target_cols, group_col).add_suffix(
                    "_{}_{}R".format(group_col, n_races)
                )
                # print(f"summarized_with:{group_col}-{summarized_with}")
                # resultsにマージ
                _results = _results.merge(
                    _summarized_with, left_on=["horse_id", group_col], right_index=True, how="left"
                )
                #print(f"mergeded considering:{group_col}-{sp_results}")
                #print(sp_results.T, sp_results[group_col])

                # 直近nレースに絞らないで過去成績をマージ
        # print(f"_horse_results:{_horse_results}")
        #_horse_results = _sort_horse_results(_horse_results)
        #print(f"SSSS{_horse_results.tail()}")

        _summarized = _summarize(_horse_results, _target_cols).add_suffix("_allR")
        print(f"summarized all:{_summarized}")
        _results = _results.merge(_summarized, left_on="horse_id", right_index=True, how="left")
        # print(f"results:{_results}")
        for group_col in _group_cols:
            # どのカテゴリ変数とともに集計しているか分かるように、列名に接尾辞をつける
            _summarized_with = _summarize_with(_horse_results, _target_cols, group_col).add_suffix(
                "_{}_allR".format(group_col)
            )
            # resultsにマージ
            _results = _results.merge(_summarized_with, left_on=["horse_id", group_col], right_index=True, how="left")
        # 前走の日付をマージ
        latest = _horse_results.groupby("horse_id")["date"].max().rename("latest")
        _results = _results.merge(latest, left_on="horse_id", right_index=True, how="left")
        # 辞書型に格納
        output_results_dict[date] = _results
    # 日付で分かれていたものを結合
    merged_data = pd.concat([output_results_dict[date] for date in output_results_dict])
    # _merged_data = merged_data
    return merged_data


def _merge_horse_info():
    """
    馬の基本情報テーブルのマージ
    """
    _merged_data = _merged_data.merge(_horse_info, left_on="horse_id", right_index=True, how="left")


def _merge_peds():
    """
    血統テーブルのマージ
    """
    _merged_data = _merged_data.merge(_peds, left_on="horse_id", right_index=True, how="left")


@property
def merged_data():
    return _merged_data


inst_R = ResultsProcessor("./data/raw/results.pkl")
_results = inst_R._preprocess()
#print(f"_results :{_results.head(1)}")
# print(_results.dtypes)

inst_RI = RaceInfoProcessor("./data/raw/race_info.pkl")
_race_info = inst_RI._preprocess()
#print(f"_race_info:{_race_info.head(1)}")
#print(_race_info.dtypes)

inst_HR = HorseResultsProcessor("./data/html/horse_results/horse_results.pkl")
_horse_results = inst_HR._preprocess()
# print(_horse_results.dtypes)
# print(f"_horse_results:{_horse_results.head(1)}")

inst_HI = HorseInfoProcessor("./data/raw/horse_info.pkl")
_horse_info = inst_HI._preprocess()
# print(f"_horse_info:{_horse_info.head(1)}")
# horse_info_processor.preprocessed_data.drop(["owner_id"], axis=1)

inst_P = PedsProcessor("./data/raw/peds.pkl")
_peds = inst_P._preprocess()
# print(f"_peds:{_peds.head(1)}")  # 血統テーブル（前処理後）


_results_after = _merge_race_info(_results, _race_info)
#print(_results_after.head(1).T)
#print(_results_after.dtypes)

# print(df.index)
column_types_dict = {
    "開催": ("float", "str", 2, True),
    "race_id": ("float", "int", 2, False),
    # "time": ("str", "datetime", 0, False),
    "days": ("float", "int", 2, False),
    "times": ("float", "int", 2, False),
    "teiryo": ("float", "int", 1, False),
    "barei": ("float", "int", 1, False),
    "kokusai": ("float", "int", 1, False),
    "shitei": ("float", "int", 1, False),
    "tokushi": ("float", "int", 1, False),
    "toku": ("float", "int", 1, False),
    "handi": ("float", "int", 1, False),
    "bettei": ("float", "int", 1, False),
    "minarai": ("float", "int", 1, False),
    "wakate": ("float", "int", 1, False),
    "shogai": ("float", "int", 1, False),
    "chiho": ("float", "int", 1, False),
    "gai": ("float", "int", 1, False),
    "kongo": ("float", "int", 1, False),
    "hinba": ("float", "int", 1, False),
    "trainer_id": ("int", "str", 5, True),
    "jockey_id": ("int", "str", 5, True),
    "owner_id": ("int", "str", 6, True),
}
df1 = convert_column_types(_results_after, column_types_dict)
# converted_results["time"] = converted_results.apply(lambda row: convert_to_datetime(row["date"], row["time"]), axis=1)
# DataFrame の各要素に関数を適用して日付と時刻の一部を取り出す
# 日付をパースして年月日の部分を取り出す
df1["date"] = pd.to_datetime(df1["date"]).dt.date
# 時刻をパースして時刻の部分を取り出す
df1["time"] = pd.to_datetime(df1["time"], format="%H:%M").dt.time
# print(df.dtypes)
#print(df1.head(1).T)
_results = df1
# print("開催", df["開催"].isna().sum())
column_types_dict = {
    "R": ("float", "int", 2, False),
    "枠番": ("float", "int", 2, False),
    "人気": ("float", "int", 2, False),
    #HorseResultsCols.PLACE: ("float", "str", 2, True),
}
df2 = convert_column_types(_horse_results, column_types_dict)
_horse_results_after = _merge_horse_results(df1, df2)
#print(_horse_results_after.head(1).T)

#print(_horse_results_after.dtypes)
# print(df.index)


separating horse results by date


  0%|          | 0/8 [00:00<?, ?it/s]

merging horse_results


  0%|          | 0/8 [00:00<?, ?it/s]

summarized all:              着順_allR     賞金_allR   着差_allR  first_corner_allR  \
horse_id                                                         
2013106055   5.585366  632.797561  0.651220           7.560976   
2015104675   5.937500  318.515625  0.784375           5.937500   
2015104707   6.473684  331.163158  0.952632           4.473684   
2016105167   6.266667  137.266667  0.990000           7.100000   
2017100316   6.466667  225.096667  0.886667           5.000000   
2017101730   6.920000  115.440000  1.420000           6.480000   
2017104302   5.758621  107.275862  1.344828           4.724138   
2017105254   7.636364  134.954545  1.540909           5.227273   
2018104865   6.480000  142.680000  1.168000           7.920000   
2018104996   7.090909  166.363636  1.554545           5.272727   
2018105996   6.250000  115.540000  1.250000           5.750000   
2019101021   6.857143   81.200000  2.085714           6.071429   
2019105392   6.200000  129.100000  1.240000           7.90000

In [201]:
# DataFrame の作成
data = {"date": ["2023-05-06 00:00:00", "2023-05-07 00:00:00"], "time": ["09:50:00", "10:30:00"]}
df = pd.DataFrame(data)


# 文字列形式の日付と時刻を datetime オブジェクトに変換する関数
def convert_to_datetime(date_str, time_str):
    # 日付と時刻を結合して datetime オブジェクトに変換
    datetime_str = date_str.split()[0] + " " + time_str
    return datetime_str


# DataFrame の各要素に関数を適用して日付と時刻の一部を取り出す
df["datetime"] = df.apply(lambda row: convert_to_datetime(row["date"], row["time"]), axis=1)
df["date"] = df["date"].apply(lambda x: x.split()[0])
df["time"] = df["time"].apply(lambda x: x.split()[0])

print(df)

         date      time             datetime
0  2023-05-06  09:50:00  2023-05-06 09:50:00
1  2023-05-07  10:30:00  2023-05-07 10:30:00


In [274]:
import pandas as pd
from abc import ABCMeta, abstractmethod
import sys
import numpy as np

sys.maxsize = sys.maxsize


class Abstract:
    def __init__(self, filepath: str):
        self.__raw_data = pd.DataFrame(pd.read_pickle(filepath))
        self.__preprocessed_data = self._preprocess()

    @abstractmethod
    def _preprocess(self):
        pass

    @property
    def raw_data(self):
        new_columns = {col: int(col) if str(col).isdigit() else col for col in self.__raw_data.columns}
        # 列名を変換
        self.__raw_data.rename(columns=new_columns, inplace=True)
        return self.__raw_data.copy()

    @property
    def preprocessed_data(self):
        return self.__preprocessed_data.copy()


inst = Abstract("./data/tmp/featured_data_20240224.pickle")
df = inst.raw_data
# new_columns = {col: int(col) if col.isdigit() else col for col in df.columns if col.isdigit()}
# 列名を変換
# df.rename(columns=new_columns, inplace=True)
pd.set_option("display.max_rows", 400)
pd.set_option("display.max_columns", 250)
print(df.head(2).T)
# DataFrame内で "XXX" を含む行と列の位置を探す
row_index, col_index = np.where(df.applymap(lambda x: "九州産馬" in str(x)))

# 結果の表示
for row, col in zip(row_index, col_index):
    print(f"Found '九州産馬' at row {row} and column {col}")

# 18列目のすべての値とその個数を取得
column_18_counts = df.iloc[:, 18].value_counts()

# 結果の表示
print(column_18_counts)

                                               1404                 1405
枠番                                                7                    5
馬番                                                9                    6
斤量                                             54.0                 54.0
単勝                                              4.5                  3.0
horse_id                                       1822                 1823
jockey_id                                         0                    1
trainer_id                                        0                    1
owner_id                                          0                    1
年齢                                                3                    3
体重                                              542                  456
体重変化                                             -2                    8
n_horses                                          1                    1
rank                                              1

/tmp/ipykernel_37206/3958479842.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  row_index, col_index = np.where(df.applymap(lambda x: "九州産馬" in str(x)))


Found '九州産馬' at row 2122 and column 18
Found '九州産馬' at row 2158 and column 18
Found '九州産馬' at row 2189 and column 18
Found '九州産馬' at row 2269 and column 18
race_condition
1       1000
2        506
3        110
九州産馬       4
Name: count, dtype: int64


In [255]:
import os

import pandas as pd

from src.constants._horse_results_cols import HorseResultsCols
from src.constants._local_paths import LocalPaths
from src.constants._master import Master
from src.preprocessing._data_merger import DataMerger


from src.preprocessing._feature_engineering import FeatureEngineering


# ****************************done*********************
def add_interval(df):
    """
    前走からの経過日数
    """
    df["date"] = pd.to_datetime(df["date"])
    df["latest"] = pd.to_datetime(df["latest"])
    df["interval"] = (df["date"] - df["latest"]).dt.days
    df.drop("latest", axis=1, inplace=True)
    return df


# ****************************done*********************
def add_agedays(df):
    """
    レース出走日から日齢を算出
    """
    # 日齢を算出
    df["birthday"] = pd.to_datetime(df["birthday"])
    df["age_days"] = (df["date"] - df["birthday"]).dt.days
    df.drop("birthday", axis=1, inplace=True)
    return df


# ****************************done*********************
def dumminize_weather(df):
    """
    weatherカラムをダミー変数化する
    """
    df = pd.Categorical(df["weather"], categories=Master.WEATHER_LIST)
    df = pd.get_dummies(df, prefix="weather_")
    return df


# ****************************done*********************
def dumminize_race_type(df):
    """
    race_typeカラムをダミー変数化する
    """
    # f["race_type"] = pd.Categorical(df["race_type"], categories=list(Master.RACE_TYPE_DICT.values()))
    # df = pd.get_dummies(df, prefix="race_type_")

    df = pd.get_dummies(
        df, columns=["race_type"], prefix="race_type", prefix_sep="_", dummy_na=False, drop_first=False, dtype=int
    )

    return df


# ****************************done*********************
def dumminize_ground_state1(df):
    """
    ground_stateカラムをダミー変数化する
    """

    df = pd.Categorical(df["ground_state1"], categories=Master.GROUND_STATE_LIST)
    df = pd.get_dummies(df, prefix="ground_state1_")

    return df


def dumminize_ground_state2(df):
    """
    ground_stateカラムをダミー変数化する
    """

    df = pd.Categorical(df["ground_state2"], categories=Master.GROUND_STATE_LIST)
    df = pd.get_dummies(df, prefix="ground_state2_")

    return df


# ****************************done*********************
def dumminize_sex(df):
    df = pd.Categorical(df["性"], categories=Master.SEX_LIST)
    df = pd.get_dummies(df, prefix="性_")
    return df


def __label_encode(df, target_col: str):
    """
    引数で指定されたID（horse_id/jockey_id/trainer_id/owner_id/breeder_id）を
    ラベルエンコーディングして、Categorical型に変換する。
    """
    # csv_path = os.path.join("./data/master/horse_id.csv")
    # ファイルが存在しない場合、空のDataFrameを作成
    csv_path = os.path.join(LocalPaths.MASTER_DIR, target_col + ".csv")
    # ファイルが存在しない場合、空のDataFrameを作成
    if not os.path.isfile(csv_path):
        target_master = pd.DataFrame(columns=[target_col, "encoded_id"])
    else:
        target_master = pd.read_csv(csv_path, dtype=object)

    # 後のmaxでエラーになるので、整数に変換
    if target_col == "horse_id":
        target_master[target_col] = target_master[target_col].astype(int)
    target_master["encoded_id"] = target_master["encoded_id"].astype(int)
    #print("target_master.:", target_master.head())
    # masterに存在しない、新しい情報を抽出
    new_target = df[[target_col]][~df[target_col].isin(target_master[target_col])].drop_duplicates(subset=[target_col])
    #print("new_target;", new_target.head())
    #print("new_target_len:", len(new_target))

    # 新しい情報を登録
    if len(target_master) > 0:
        new_target["encoded_id"] = [i + max(target_master["encoded_id"]) for i in range(1, len(new_target) + 1)]
        # 整数に変換
        new_target["encoded_id"] = new_target["encoded_id"].astype(int)
    else:  # まだ1行も登録されていない場合の処理
        new_target["encoded_id"] = [i for i in range(len(new_target))]
    # インデックスをリセット
    new_target.reset_index(drop=True, inplace=True)

    # 元のマスタと繋げる
    # new_target_master = pd.concat([target_master, new_target]).set_index(target_col)["encoded_id"]
    new_target_master = pd.concat([target_master, new_target], ignore_index=True).reset_index(drop=True)

    # マスタファイルを更新
    new_target_master.to_csv("./data/master/horse_id.csv", index=False)
    # ラベルエンコーディング実行
    # new_target_master から target_col と encoded_id の列のみを抽出
    mapping_df = new_target_master[[target_col, "encoded_id"]]

    # df と new_target_master を結合して、target_col に対応する encoded_id を取得
    merged_df = df.merge(mapping_df, on=target_col, how="left")

    # 新しい列を作成し、結合された encoded_id を代入
    df[str(target_col) + "_cat"] = pd.Categorical(merged_df["encoded_id"])

    return df


def encode_horse_id(df):
    """
    horse_idをラベルエンコーディングして、Categorical型に変換する。
    """
    __label_encode(df, "horse_id")
    return df


def encode_jockey_id(df):
    """
    jockey_idをラベルエンコーディングして、Categorical型に変換する。
    """
    __label_encode(df, "jockey_id")
    return df


def encode_trainer_id(df):
    """
    trainer_idをラベルエンコーディングして、Categorical型に変換する。
    """
    __label_encode(df, "trainer_id")
    return df


def encode_owner_id(df):
    """
    owner_idをラベルエンコーディングして、Categorical型に変換する。
    """
    __label_encode(df, "owner_id")
    return df


def encode_breeder_id(df):
    """
    breeder_idをラベルエンコーディングして、Categorical型に変換する。
    """
    __label_encode(df, "breeder_id")
    return df


# ****************************done*********************
def dumminize_kaisai(df):
    """
    開催カラムをダミー変数化する
    """
    # df[HorseResultsCols.PLACE] = pd.Categorical(df[HorseResultsCols.PLACE], categories=list(Master.PLACE_DICT.values()))
    # df = pd.get_dummies(df, prefix=HorseResultsCols.PLACE)

    df = pd.get_dummies(
        df,
        columns=[HorseResultsCols.PLACE],
        prefix=HorseResultsCols.PLACE,
        prefix_sep="_",
        dummy_na=False,
        drop_first=False,
        dtype=int,
    )

    return df


# ****************************done*********************
def dumminize_around(df):
    """
    aroundカラムをダミー変数化する
    """
    df = pd.Categorical(df["around"], categories=Master.AROUND_LIST)
    df = pd.get_dummies(df, prefix="around_")
    return df


# ****************************done*********************
def dumminize_race_class(df):
    """
    race_classカラムをダミー変数化する
    """
    df = pd.Categorical(df["race_class"], categories=Master.RACE_CLASS_LIST)
    df = pd.get_dummies(df, prefix="race_class_")
    return df


df = pd.read_csv("./data/tmp/for_sandbox/test_df.csv")
# print("df:", df)
# target_col = "horse_id"
# print("df:", df)
# print("df:df.head()")
# カテゴリ変数の処理

# add_interval(df)
# print("interval:", df["interval"].head())

# add_agedays(df)
# print("age_days:", df["age_days"].head())
df = dumminize_ground_state1(df)

if any(col.startswith("ground_state") for col in df.columns):
#"ground_state_d" で始まる列を選択して表示
    ground_state1_cols = df.filter(like="ground_state")
    print(ground_state1_cols.head())
else:
    print("No columns starting with 'ground_state' found.")

df2 = dumminize_ground_state2(df)

if any(col.startswith("ground_state") for col in df.columns):
# "ground_state_d" で始まる列を選択して表示
    ground_state2_cols = df2.filter(like="ground_state")
    print(ground_state2_cols.head())
else:
    print("No columns starting with 'ground_state' found.")


#df = dumminize_race_type(df)
#if any(col.startswith("race_type_") for col in df.columns):
    # "ground_state_d" で始まる列を選択して表示
#    ground_state_d_cols = df.filter(like="race_type_")
#    print(ground_state_d_cols.head())
#else:
#    print("No columns starting with 'race_type' found.")

# df = dumminize_sex(df)

# if any(col.startswith("性_") for col in df.columns):
# "ground_state_d" で始まる列を選択して表示
#    ground_state_d_cols = df.filter(like="性_")
#    print(ground_state_d_cols.head())
# else:
#    print("No columns starting with '性_' found.")

# df = dumminize_weather(df)
# if any(col.startswith("weather_") for col in df.columns):
# "ground_state_d" で始まる列を選択して表示
#    ground_state_d_cols = df.filter(like="weather_")
#    print(ground_state_d_cols.head())
# else:
#    print("No columns starting with 'weather_' found.")

# encode_horse_id(df)
# target_col = "horse_id"
# print("cat+;", cat.head())


# encode_jockey_id(df)
# target_col = "jockey_id"
# print("jockey_id", df["jockey_id"].head())
# encode_trainer_id(df)
# target_col = "trainer_id"
# print("trainer_id", df["trainer_id"].head())
# encode_owner_id(df)
# target_col = "owner_id"
# print("owner_id", df["owner_id"].head())
# encode_breeder_id(df)
# target_col = "breeder_id"
# print("breeder_id", df["breeder_id"].head())


# df = dumminize_kaisai(df)
# if any(col.startswith("開催_") for col in df.columns):
# "ground_state_d" で始まる列を選択して表示
#    ground_state_d_cols = df.filter(like="開催_")
#    print(ground_state_d_cols.head())
# else:
#    print("No columns starting with 開催_' found.")

# df = dumminize_around(df)
# if any(col.startswith("around_") for col in df.columns):
# "ground_state_d" で始まる列を選択して表示
#    ground_state_d_cols = df.filter(like="around_")
#    print(ground_state_d_cols.head())
# else:
#    print("No columns starting with 'around_' found.")
# print("around", df["around"].head())
# df=dumminize_race_class(df)
# if any(col.startswith("race_class") for col in df.columns):
# "ground_state_d" で始まる列を選択して表示
#    ground_state_d_cols = df.filter(like="race_class_")
#    print(ground_state_d_cols.head())
# else:
#    print("No columns starting with 'race_class_' found.")
# print("all done", df.head())

   ground_state1__良  ground_state1__稍重  ground_state1__重  ground_state1__不良
0              True              False             False              False
1              True              False             False              False
2              True              False             False              False
3              True              False             False              False
4              True              False             False              False


KeyError: 'ground_state2'

In [256]:
# horse_sortes = horse.iloc[:, 0].sort_values()
# 重複行を含むすべての行をマークします
# duplicated_rows = horse_sortes[horse_sortes.duplicated(keep=False)]
# csv_path = os.path.join(LocalPaths.MASTER_DIR, target_col + ".csv")
# ファイルが存在しない場合、空のDataFrameを作成


def __label_encode(df, target_col: str):
    """
    引数で指定されたID（horse_id/jockey_id/trainer_id/owner_id/breeder_id）を
    ラベルエンコーディングして、Categorical型に変換する。
    """
    csv_path = os.path.join("./data/master/horse_id.csv")
    # ファイルが存在しない場合、空のDataFrameを作成
    if not os.path.isfile(csv_path):
        target_master = pd.DataFrame(columns=[target_col, "encoded_id"])
    else:
        target_master = pd.read_csv(csv_path, dtype=object)

    # 後のmaxでエラーになるので、整数に変換
    if target_col == "horse_id":
        target_master[target_col] = target_master[target_col].astype(int)
    target_master["encoded_id"] = target_master["encoded_id"].astype(int)
    print("target_master.:", target_master.head())
    # masterに存在しない、新しい情報を抽出
    new_target = df[[target_col]][~df[target_col].isin(target_master[target_col])].drop_duplicates(subset=[target_col])
    print("new_target;", new_target.head())
    print("new_target_len:", len(new_target))
    # 新しい情報を登録
    if len(target_master) > 0:
        new_target["encoded_id"] = [i + max(target_master["encoded_id"]) for i in range(1, len(new_target) + 1)]
        # 整数に変換
        new_target["encoded_id"] = new_target["encoded_id"].astype(int)
    else:  # まだ1行も登録されていない場合の処理
        new_target["encoded_id"] = [i for i in range(len(new_target))]
    # インデックスをリセット
    new_target.reset_index(drop=True, inplace=True)

    # 元のマスタと繋げる
    # new_target_master = pd.concat([target_master, new_target]).set_index(target_col)["encoded_id"]
    new_target_master = pd.concat([target_master, new_target], ignore_index=True).reset_index(drop=True)

    # マスタファイルを更新
    new_target_master.to_csv(csv_path, index=False)
    # ラベルエンコーディング実行
    df[target_col] = pd.Categorical(df[target_col].map(new_target_master[target_col]))
    return new_target, new_target_master


df = pd.read_csv("./data/tmp/for_sandbox/test_df.csv")
target_col = "horse_id"

#new_target, new_target_master = __label_encode(df, target_col)

print("df:",df)
#print("NTM;", new_target_master.head())
#print("NTM_len:", len(new_target_master))

df:       Unnamed: 0  枠番  馬番    斤量     単勝    horse_id  jockey_id  trainer_id  \
0           1404   7   9  54.0    4.5  2020103617       5339        1059   
1           1405   5   6  54.0    3.0  2020105055       1170        1048   
2           1406   1   1  56.0    9.7  2020103452       5509        1017   
3           1407   6   8  54.0   11.8  2020101201       1091        1167   
4           1408   5   5  54.0   67.3  2020101564       1134        1038   
...          ...  ..  ..   ...    ...         ...        ...         ...   
3709        3739   4   5  56.0   15.6  2019104703       1178        1155   
3710        3740   2   2  53.0   17.9  2020103009       1093        1062   
3711        3741   4   6  56.0  175.8  2018102204       1018        1085   
3712        3742   3   3  56.0  137.7  2015106165       1157        1104   
3713        3743   5   8  56.0   85.5  2019104607       1025        1116   

      owner_id  性  年齢   体重  体重変化  n_horses  rank       race_id place  days  \
0    

In [50]:
import pandas as pd

# サンプルのDataFrameを作成します
data = {"horse_id": ["A", "B", "C", "C", "D"], "value": [1, 2, 3, 4, 5]}
df = pd.DataFrame(data)

# horse_id列をインデックスとして設定します
df.set_index("horse_id", inplace=True)

print(df)

          value
horse_id       
A             1
B             2
C             3
C             4
D             5


In [248]:
df = pd.read_csv("./data/tmp/for_sandbox/test_df.csv")
df["開催"].head()

0    6
1    6
2    6
3    6
4    6
Name: 開催, dtype: int64

In [281]:
import pandas as pd

from src.preprocessing._abstract_data_processor import AbstractDataProcessor
from src.preprocessing._feature_engineering import FeatureEngineering
from src.training._keiba_ai_factory import KeibaAIFactory


class ReturnProcessor(AbstractDataProcessor):
    def __init__(self, filepath):
        """
        初期処理
        """
        super().__init__(filepath)

    def _preprocess(df):
        """
        前処理
        """

        return_dict = {}
        return_dict["tansho"] = df.__tansho()
        return_dict["fukusho"] = df.__fukusho()
        return_dict["umaren"] = df.__umaren()
        return_dict["umatan"] = df.__umatan()
        return_dict["wide"] = df.__wide()
        return_dict["sanrentan"] = df.__sanrentan()
        return_dict["sanrenpuku"] = df.__sanrenpuku()
        return return_dict

    def __tansho(df):
        """
        単勝
        """
        tansho = df.raw_data[df.raw_data[0] == "単勝"][[1, 2]]
        tansho.columns = ["win", "return"]

        for column in tansho.columns:
            tansho[column] = pd.to_numeric(tansho[column], errors="coerce")

        return tansho


inst = ReturnProcessor("models/20240224/basemodel_2023_2024.pickle")
pd = inst._preprocess()

pd

AttributeError: 'KeibaAI' object has no attribute 'columns'

In [292]:
import pandas as pd

from src.preprocessing._abstract_data_processor import AbstractDataProcessor


class ReturnProcessor(AbstractDataProcessor):
    def __init__(self, filepath):
        """
        初期処理
        """
        super().__init__(filepath)
    def _preprocess(self):
       pass

    def __tansho(self):
        tansho = self.raw_data[self.raw_data[0] == "単勝"][["race_id", 1, 2]]
        tansho.columns = ["race_id", "win", "return"]
        tansho.set_index("race_id", inplace=True)
        return tansho


rt = ReturnProcessor("./data/html/race_return/return_tables.pkl")
rt.__tansho()

AttributeError: 'ReturnProcessor' object has no attribute '__tansho'

In [293]:
import pandas as pd

# サンプルデータ
data = {"column_with_br": ["5 - 8br5 - 6", "6 - 8br5 - 6br7 - 9", "4 - 7br", "3 - 5br4 - 6br5 - 7br6 - 8"]}

# DataFrame作成
df = pd.DataFrame(data)

# "br"で分割し、expand=Trueを指定してDataFrameに展開する
wins = df["column_with_br"].str.split("br", n=4, expand=True)

# 不足する部分を0に置き換える
wins = wins.fillna(0)

# 列名を変更する
wins.columns = [f"win_{i}" for i in range(len(wins.columns))]

print(wins)

   win_0  win_1  win_2  win_3
0  5 - 8  5 - 6      0      0
1  6 - 8  5 - 6  7 - 9      0
2  4 - 7             0      0
3  3 - 5  4 - 6  5 - 7  6 - 8


In [333]:
%autoreload

UsageError: Line magic function `%autoreload` not found.


In [342]:
from itertools import permutations

from scipy.special import comb

from src.preprocessing._return_processor import ReturnProcessor


# class BettingTickets:
#    """
#    馬券の買い方と、賭けた時のリターンを計算する。
#    """

rt = ReturnProcessor("./data/html/race_return/return_tables.pkl")
__returnTables = rt._preprocess()
__returnTablesTansho = __returnTables["tansho"]
##self.__returnTablesFukusho = self.__returnTables["fukusho"]
# self.__returnTablesUmaren = self.__returnTables["umaren"]
# self.__returnTablesUmatan = self.__returnTables["umatan"]
# self.__returnTablesWide = self.__returnTables["wide"]
# self.__returnTablesSanrenpuku = self.__returnTables["sanrenpuku"]
# self.__returnTablesSanrentan = self.__returnTables["sanrentan"]


def calculate_hit(umaban, race_id, __returnTablesTansho):
    hit = 0
    table_1R = __returnTablesTansho[__returnTablesTansho.index == race_id]
    i = 0
    while i < len(umaban):
        win_column = f"win_{i}"
        if win_column in table_1R.columns and table_1R[win_column].iloc[0] != 0:
            return_column = f"return_{i}"
            hit += table_1R[return_column].iloc[0]
        i += 1
    return hit


def bet_tansho(race_id: str, umaban: list, amount: float):
    """
    race_id: レースid。
    umaban: 賭けたい馬番をリストで入れる。一頭のみ賭けたい場合もリストで入れる。
    amount: 1枚に賭ける額。
    """
    # 賭ける枚数

    n_bets = len(umaban)
    # print(f"n_bets:{n_bets}")
    if n_bets == 0:
        return 0, 0, 0
    else:
        # 賭けた合計額
        bet_amount = n_bets * amount
        # print(f"bet_amount:{bet_amount}")
        # 賭けるレースidに絞った単勝の払い戻し表
        # print("self.__returnTables", self.__returnTables)
        # 的中判定
        return_amount = 0
        table_1R = __returnTablesTansho[__returnTablesTansho.index == race_id]
        length = len(table_1R.columns)
        print(f"i:{length}")
        for uma in umaban:
            for i in range(length // 2):
                win_column = f"win_{i}"
                if table_1R.loc[race_id, win_column] != 0 and table_1R.loc[race_id, win_column] == uma:
                    return_column = f"return_{i}"
                    return_amount += table_1R[return_column] * amount / 100
        return n_bets, bet_amount, return_amount


print(__returnTablesTansho.head())
print(__returnTablesTansho.dtypes)
__returnTablesTansho[__returnTablesTansho.index == 202301010101]
print(bet_tansho(202301010101, [5, 6, 7], 1))

単勝列数:2
枠連列数:2
複勝列数:4
馬連列数:2
馬単列数:2
ワイド列数:5
三連単列数:2
三連複列数:2
単勝列数:2
枠連列数:2
複勝列数:4
馬連列数:2
馬単列数:2
ワイド列数:5
三連単列数:2
三連複列数:2
             win_0 win_1  return_0  return_1
race_id                                     
202301010101     5     0       120         0
202301010102     9     0       240         0
202301010103    11     0       840         0
202301010104     9     0      1970         0
202301010105     4     0       460         0
win_0       object
win_1       object
return_0     int64
return_1     int64
dtype: object
i:4
(3, 3, 0)
